# LangChain 1.x 系で熊剣迷路問題

(Version: 0.0.15.4)

## 概要

熊剣迷路問題の経験を元に作ったエージェントフレームワークの MemoryBanditWorkflow。それを LangChain 1.x 系に対応させ、さらにはやりの skills やツールボックスなどの導入に相当する subtool_do の導入も行った。

## MemoryBanditWorkflow のアイデア

最近 LLM についてコンテクストエンジニアリングという言葉をよく聞く。実は Web でなく API から使う LLM は「ステートレス」で、会話などの記憶はデフォルトではしてくれない。会話を続けようと思ったら、メッセージをユーザー側で記録しておき、それをいちいち API に与えないといけないのだ。古い記憶を呼び戻したりするには、単純に記録するだけでなく検索基盤も必要となる。そのような基盤をひっくるめてコンテクストと一般的に呼んでおり、それをどううまく構築するかが問題となっている。

特にコンテクストのうち、記憶の基盤は単に「メモリ」と呼ばれている。PC のメモリなどとごっちゃになって、わかりにくくもあるのだが、そう呼ばれているのだからしかたがない。

メモリ機能をなかなか使ってくれないのに業を煮やして、「バンディットに特定のツールを登録するツールみたいなのを用意して、どういうツールを強制して数を増やして欲しいかを AI さん自身が決められる」というバンディット機能を作って、メモリを使うのを強制してみたのが前回。今回はそれを拡張して、ワークフローも実行させられるようにしてみた。

そのアイデアの軌跡は↓にある。

\[cocolog:95619779\](2025年9月)  
《「LLM のメモリ機能を強制的に使うバンディットマシンの試験実装」と「LLM のメモリ機能とバンディット機能の試験実装」を行った。後者がメインの成果物で、メモリ機能の使用増加をどう強制するかから拡張したフレームワーク。 - JRF のひとこと》  
http://jrf.cocolog-nifty.com/statuses/2025/09/post-8225e2.html

基本的にはこれまでの熊剣迷路問題を使い、メモリ機能とバンディット機能と今回新しいワークフロー機能を使う。

簡単な迷路問題ではあるが、かなり本格的なメモリ機能・バンディット機能・ワークフロー機能を用意している。

セマンティックサーチとかを用意するのは、めんどくさかったので、そういうバックエンドはこれまた AI さんに「データベース偽装」をしてもらっている。

なお、熊剣迷路問題は↓からたどっていただきたい。

《JRF-2018/langchain_maze: 熊剣迷路問題 revisited》  
https://github.com/JRF-2018/langchain_maze

ところで、今回のような手法はすでにどこかにあるとは思う。ワークフローというのは手垢のついた考え方だ。新規性がある話ではあまりないのかもしれないが、今回の試みが、何かの参考になれば幸いである。


## サブツールというアイデア

Claude Code はすでに AGI じゃないかと話題だが、私の MemoryBanditWorkflow のようにメモリの使い方にハーネス＝制約を効かせる実装はどうするのだろうか？

おそらく、それは Claude Code にサブエージェント的なものとしてメモリのハーネスの効いたエージェントを実装させることになるのだろう。そういう Claude Code に書かせたものとして、MemoryBanditWorkflow のような枠組みは有効であると思われる。

Claude Code などでは、skills というものがある。これは、MCP などのツール類の情報が肥大化し、それを常にコンテクストに置いておくのは効率が悪いという判断のもと、必要なときに必要なスキルをスキルツリーから「探索」する形で置いておくというアイデアだ…というのが私の認識である。

このようなスキルも MemoryBanditWorkflow に取り込みたい。どうすればいいのだろうか？

Claude Code のスキルでは、Claude Code がシェルや Python コードを実行できることを前提として、コマンドやコードが示される。しかし、シェルや REPL は、ハーネスを効かせたエージェントでは、一般に使わせることなど考えられない。しかし、そうでないとすれば、ツールをすべて用意しておくしかないが、そうすると、コンテクストを「汚染」してしまう。

そこでひらめいたのが、サブツールという考え方である。ツールとしては subtool_do だけがコンテクストに載っていて、どういうサブツールが使えるかは subtool_show で、まるで SKILL.md を読むように後から適宜知る必要がある…とすればいいのではないかと考えた。無限の拡張性をもたせた上での「オンデマンド学習」というわけである。

問題はそれを LLM さん(Gemini さん)が理解してくれるかどうかである。

このあたりのアイデアの軌跡は↓にある。

\[cocolog:95822546\](2026年1月)  
《MemoryBanditWorkflow (参: \[cocolog:95619779\](2025年9月)) を Claude Code などの SKILL.md などを利用して実現するにはどうすればいいか。試みに Claude さん自身に聞いてみた。 - JRF のひとこと》  
http://jrf.cocolog-nifty.com/statuses/2026/01/post-b86e58.html

## 前回へのリンク

《experimental_bandit_workflow_0_0_2.ipynb - JRF-2018/langchain_maze》  
https://github.com/JRF-2018/langchain_maze/blob/master/experimental_bandit_workflow_0_0_2.ipynb


## 前回からの変更点

LangChain 1.x 系に対応した。まず、Pydantic v2 と Gemini の実装のクセらしく型エラーへの対処が難しかったが、なんとか今は動くようになっている。しかし、弥縫策で、この先はどうなるか自信がない。

ツールを格納して必要なツールのみ説明を読んで使うための subtool_do や subtool_show を導入した。subtool_show で表示されるのがおよそ他のフレームワークでは SKILL.md で書かれることに相当する。


## 結論

今回はコストがかかるのを嫌ってはじめからゴールさせるつもりはなく、ワークフローを一度使ったところで実験終了させた。テストは gemini-2.5-flash-lite さんで行い、最終的には gemini-3-flash-preview さんに実行をお願いした。

flash-lite さんは、サブツールの実行はうまくいかないこともあったが、gemini-3 さんは、ちゃんと実行できたようだ。サブツールはまずまずうまくいったというのが私の評価である。

なお、再現性がないため、最初からの再実行はやっていないのはご容赦願いたい。コストもかかるしね \(^^;。


## 著者

JRF ( http://jrf.cocolog-nifty.com/statuses , Twitter (X): @jion_rockford )


## ライセンス

基本短いコードなので(私が作った部分は)パブリックドメインのつもりです。気になる方は MIT License として扱ってください。

かなり AI さん達(Gemini さんや ChatGPT さんや Claude さんや Grok さん)に教わって作っています。


## 実装

まず、必要なライブラリを読み込みます。

In [1]:
!pip install -q -U langchain langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.1/500.1 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.1/158.1 kB 3.4 MB/s eta 0:00:00


Gemini にアクセスします。シークレットで Gemini API キーを Google AI Studio からインポートすると GOOGLE_API_KEY というシークレットができるはずです。それを使います。

In [2]:
import os
from langchain.chat_models import init_chat_model
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from google.colab import userdata

#os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

llm = init_chat_model(
    "google_genai:gemini-3-flash-preview",
    google_api_key=userdata.get('GOOGLE_API_KEY'),
#    thinking_level="low", # for gemini-3.0
#    thinking_budget=0, # for gemini-2.5
)
emb_llm = GoogleGenerativeAIEmbeddings(
    model='gemini-embedding-001',
    google_api_key=userdata.get('GOOGLE_API_KEY'),
)


ちゃんと Gemini にアクセスできるかテストします。

In [3]:
import os
from langchain_core.messages import HumanMessage

# Gemini 3 から次のようなヘルパー関数が必要となった。
def get_content_text(content):
    if isinstance(content, list):
        texts = []
        for item in content:
            if isinstance(item, dict):
                if item.get('type') == 'text':
                    texts.append(item.get('text', ''))
            elif isinstance(item, str):
                texts.append(item)
        return "".join(texts)
    return content

response = llm.invoke([HumanMessage(content="Geminiモデルの特徴を教えてください")])
print(get_content_text(response.content))


Googleが開発した最新のAIモデルである「Gemini（ジェミニ）」には、他のAI（GPT-4など）と比較して際立った特徴がいくつかあります。

主な特徴を5つのポイントに分けて解説します。

---

### 1. ネイティブ・マルチモーダル
最も大きな特徴は、最初から**「マルチモーダル」**として設計されている点です。
*   **どういうことか：** 従来のAIは、テキスト用、画像用、音声用といった別々のモデルを組み合わせることが多かったのですが、Geminiは一つのモデルでテキスト、画像、音声、動画、プログラミングコードを同時に理解し、処理できます。
*   **メリット：** 動画の内容を読み取って要約したり、手書きの図からコードを生成したりする能力が非常に高いです。

### 2. 圧倒的な「長文コンテキストウィンドウ」
Gemini 1.5 Proなどのモデルでは、一度に処理できる情報の量（コンテキストウィンドウ）が非常に大きいです。
*   **容量：** 最大100万〜200万トークン以上を一度に読み込めます。
*   **できること：** 数時間の動画、数千行のソースコード、あるいは数冊分の分厚い本を一気に読み込ませ、その内容について質問したり分析させたりすることが可能です。これは競合モデル（GPT-4oなど）と比較してもトップクラスの性能です。

### 3. 用途に合わせたモデル展開
Geminiは、利用シーンに合わせて複数のサイズが用意されています。
*   **Gemini Ultra：** 最も複雑なタスク向け。高度な推論、科学的タスク、コーディングに特化。
*   **Gemini Pro：** 性能と速度のバランスが良い汎用モデル。Google AI StudioやGeminiのチャットサービスで主に使われています。
*   **Gemini Flash：** 応答速度が非常に速く、軽量なモデル。コスト効率が良く、大量の処理を素早く行いたい開発者向け。
*   **Gemini Nano：** デバイス上で動作するように設計された最小モデル。インターネットに繋がなくても、スマートフォン（PixelやGalaxyなど）上で直接AIが動作します。

### 4. Googleエコシステムとの連携
Googleの製品群と深く統合され

埋め込みベクトルも試しておきます。

In [4]:
emb_llm.embed_query("これはテストです。")[:5]

[-0.0140901245, -0.006826138, 0.0076896483, -0.07491117, 0.011200756]

基本的なモジュールを読み込みます。

In [5]:
import os
import math
import numpy as np
import random
import re
from pprint import pprint
from time import sleep
import pickle
np.set_printoptions(legacy='1.25')

セーブ／ロードのために次のコードを実行します。

In [6]:
PLAY_GAME_SAVE = "langchain_maze.pickle"

ゲームのメインオブジェクト。ごく簡単な迷路というかダンジョンというか…。

In [7]:
class Game:
    initial_map = """\
■■■■■■■■■
■■■■■■■Ｇ■
■□□□□□■□■
■□■■■□□□■
■□■■■■■■■
■◎■■■■■△■
■□■■■■■□■
■□□□□□□□■
■■■■Ｓ■■■■
■■■■■■■■■
"""

    def __init__ (self, initial_map=None, hint=True):
        if initial_map is not None:
            self.initial_map = initial_map
        map = self.initial_map
        self.map = map
        self.written_map = re.sub("[◎△]", "？", map)
        l = map.splitlines(True)
        self.map_size = (len(l[0]) - 1, len(l))
        self.hint = hint
        self.actions = {
            "上に行く": self.move_up,
            "下に行く": self.move_down,
            "左に行く": self.move_left,
            "右に行く": self.move_right,
            "熊を殺す": self.fight,
            "物を取る": self.get_sword,
            "何もしない": self.do_nothing,
        }
        self.pos = self.get_start_pos()
        self.sword = False
        self.bear_killed = 0
        self.goal = False
        self.prev_killed = False
        self.kill_hint = False
        self.prev_success = True

    def read_map (self, p):
        x = p[0]
        y = p[1]
        if x < 0 or x >= self.map_size[0]\
           or y < 0 or y >= self.map_size[1]:
            return "■"
        else:
            l = self.map.splitlines(True)
            return l[y][x]

    def set_map (self, pos, ch):
        idx = pos[1] * (self.map_size[0] + 1) + pos[0]
        self.map = self.map[:idx] + ch + self.map[idx + 1:]

    def get_pos (self, ch, written=False):
        if written:
            map = self.written_map
        else:
            map = self.map
        r = []
        for p in [i for i in range(len(map)) if map.startswith(ch, i)]:
            y = p // (self.map_size[0] + 1)
            x = p % (self.map_size[0] + 1)
            r.append(np.array([x, y]))
        return r

    def get_start_pos (self):
        return self.get_pos("Ｓ")[0]

    def read_neighbors (self):
        c = self.read_map(self.pos)
        cu = self.read_map(self.pos + np.array([0, -1]))
        cd = self.read_map(self.pos + np.array([0, +1]))
        cl = self.read_map(self.pos + np.array([-1, 0]))
        cr = self.read_map(self.pos + np.array([+1, 0]))
        return [c, cu, cd, cl, cr]

    def change_neighbors(self, from_ch, to_ch):
        for d in [[0, 0], [0, -1], [0, +1], [-1, 0], [+1, 0]]:
            p = self.pos + np.array(d)
            c = self.read_map(p)
            if c == from_ch:
                self.set_map(p, to_ch)

    def move (self, res, d):
        self.prev_killed = False
        self.prev_success = False
        c = self.read_map(self.pos + d)
        if c == "◎":
            self.prev_killed = True
            self.pos = self.get_start_pos()
            return "熊を無視して進もうとしたが、熊に殺された。" \
                + "スタート地点で復活。"
        if c == "■":
            return "壁があって進めない。"

        self.prev_success = True
        self.pos += d
        if c == "Ｇ":
            self.goal = True
            return "ゴール！ ゲームクリア。"

        nb = self.read_neighbors()
        ad = ""
        if "◎" in nb:
            ad += "熊に出会った。"
        if "△" in nb:
            ad += "近くに剣がある。剣を取ることができる。"
        return res + ad

    def move_up (self):
        return self.move("上に進んだ。", np.array([0, -1]))

    def move_down (self):
        return self.move("下に進んだ。", np.array([0, +1]))

    def move_left (self):
        return self.move("左に進んだ。", np.array([-1, 0]))

    def move_right (self):
        return self.move("右に進んだ。", np.array([+1, 0]))

    def fight (self):
        self.prev_success = False
        self.prev_killed = False
        if "◎" in self.read_neighbors():
            if self.sword:
                self.prev_success = True
                self.change_neighbors("◎", "□")
                self.bear_killed += 1
                return "熊を倒した！"
            else:
                self.pos = self.get_start_pos()
                self.prev_killed = True
                if self.hint:
                    self.kill_hint = True
                    return "熊に敗れ殺された。剣があれば勝てたかもしれない。" \
                        + "スタート地点で復活。"
                else:
                    return "熊に敗れ殺された。スタート地点で復活。"
        return "無意味な指示。敵がいない。"

    def get_sword (self):
        self.prev_killed = False
        self.prev_success = False
        if "△" in self.read_neighbors():
            self.sword = True
            self.prev_success = True
            self.change_neighbors("△", "□")
            return "剣を取った。"
        return "無意味な指示。近くに取れる物がない。"

    def do_nothing (self):
        self.prev_killed = False
        self.prev_success = False
        return "無意味な指示。"

    def available_actions (self):
        nb = self.read_neighbors()
        l = []
        if nb[1] != "■":
            l.append("上に行く")
        if nb[2] != "■":
            l.append("下に行く")
        if nb[3] != "■":
            l.append("左に行く")
        if nb[4] != "■":
            l.append("右に行く")
        if "△" in nb:
            l.append("物を取る")
        if "◎" in nb:
            l.append("熊を殺す")
        return l

    def surroundings (self):
        x = self.pos[0]
        y = self.pos[1]
        return \
            "".join(["".join([self.read_map(np.array([i, j]))
                              if i != x or j != y else "▼"
                              for i in range(x - 2, x + 3)])
                     + "\n"
                     for j in range(y - 2, y + 3)])


In [8]:
def flip_text_map (m):
    return "\n".join([s[::-1] for s in m.splitlines()] + [""])

def rotate_text_map (m):
    m = list(m.splitlines())
    return "\n".join(["".join([m[len(m) - j - 1][i] for j in range(len(m))])
                      for i in range(len(m[0]))] + [""])


def search_path (game, from_pos, to_pos, visit=None):
    if visit is None:
        visit = set()
    visit.add(tuple(from_pos))
    if tuple(from_pos) == tuple(to_pos):
        return (tuple(from_pos), [])
    if game.read_map(from_pos) == "■":
         return None
    r = []
    for p in [(from_pos[0], from_pos[1] - 1),
              (from_pos[0], from_pos[1] + 1),
              (from_pos[0] - 1, from_pos[1]),
              (from_pos[0] + 1, from_pos[1])]:
        if p not in visit and game.read_map(p) != "■":
            q = search_path(game, p, to_pos, visit.copy())
            if q:
                r.append(q)
    if r:
        return (tuple(from_pos), r)
    return None


ゲームがうまく動くかテスト。

In [ ]:
game = Game()

In [ ]:
print(game.surroundings())

■■■■■
□□□□□
■■▼■■
■■■■■
■■■■■



In [ ]:
print(game.move_up())
print(game.surroundings())

上に進んだ。
■■■■■
■■■■■
□□▼□□
■■Ｓ■■
■■■■■



In [ ]:
print(game.move_left())
print(game.surroundings())

左に進んだ。
◎■■■■
□■■■■
□□▼□□
■■■Ｓ■
■■■■■



In [ ]:
print(game.move_right())
print(game.surroundings())

右に進んだ。
■■■■■
■■■■■
□□▼□□
■■Ｓ■■
■■■■■



In [ ]:
print(game.move_down())
print(game.surroundings())

下に進んだ。
■■■■■
□□□□□
■■▼■■
■■■■■
■■■■■



In [ ]:
print(game.fight())
print(game.surroundings())

無意味な指示。敵がいない。
■■■■■
□□□□□
■■▼■■
■■■■■
■■■■■



In [ ]:
print(game.get_sword())
print(game.surroundings())

無意味な指示。近くに剣がない。
■■■■■
□□□□□
■■▼■■
■■■■■
■■■■■



LLM を使いながらゲームを解くクラスを実装していきます。まずは必要なライブラリから。

In [9]:
from pydantic import ValidationError
from typing import List, Dict, Any, Tuple, Union
from textwrap import dedent
import datetime
import copy
import inspect

# LangChainのコンポーネントをインポート
from langchain_core.tools import tool, Tool
from langchain.agents.middleware import SummarizationMiddleware
from langchain.agents.middleware.summarization import DEFAULT_SUMMARY_PROMPT
from langchain.agents import create_agent
#from langgraph.prebuilt import create_react_agent
#from langchain_core.messages.utils import count_tokens_approximately
#from langgraph.prebuilt.chat_agent_executor import AgentState
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.prompts.chat import ChatPromptTemplate
#from langmem.short_term import SummarizationNode, summarize_messages
from langchain_core.messages import AIMessage, ToolMessage, HumanMessage, SystemMessage
from langgraph.errors import GraphRecursionError

SUMMARY_PROMPT = DEFAULT_SUMMARY_PROMPT + "\n\n**要約は日本語でしてください。**"


In [10]:
def calc_embedding_variance(embeddings):
    if not embeddings or len(embeddings) < 2:
        return 0.0

    embeddings_array = np.array(embeddings)
    mean_vector = np.mean(embeddings_array, axis=0)
    squared_distances = np.linalg.norm(embeddings_array - mean_vector, axis=1)**2
    variance = np.mean(squared_distances)

    return variance

def short_repr(x, max_len=80):
    repr_str = repr(x)

    if len(repr_str) > max_len:
        ellipsis_len = 3

        head_len = max_len - ellipsis_len - 1
        tail_len = 1
        return repr_str[:head_len] + "..." + repr_str[-tail_len:]
    else:
        return repr_str

def get_content_text(content):
    if isinstance(content, list):
        texts = []
        for item in content:
            if isinstance(item, dict):
                if item.get('type') == 'text':
                    texts.append(item.get('text', ''))
            elif isinstance(item, str):
                texts.append(item)
        return "".join(texts)
    return content


MemoryBanditWorkflow は以前は PlayGame と一つだったが、汎用性をもたせるため分けました。汎用性があることは experimental_rag_0.0.16.ipynb などでも実証済み。いろいろ入れて、とても長大で複雑な実装になっています。すみません。

In [18]:
class MemoryBanditWorkflow:
    def __init__ (self, llm=llm, llm2=llm, emb_llm=emb_llm,
                  save_file=None):
        self.llm = llm
        self.llm2 = llm2
        self.emb_llm = emb_llm
        self.save_file = save_file

        self.core_context = ""
        self.plan = "まだ計画と方針はセットされていません。"
        self.scratchpad = ""

        self.messages = []
        self.running_summary = None
        self.system_prompt = """\
これはメモリ機能とバンディット機能の動きを見るための実験です。
現在の計画と方針と周囲の状況を考慮し、必要に応じて計画と方針を更新してください。
あなたとは別の者が次の行動をしやすいよう計画と方針を残してください。
実験にふさわしく、できる限り、ツールを利用し、特に、メモリを検索し、その文書を更新して、後のコマンドに備えてください。
メモリの id は memory:～ という形式です。memory:5555 を指定するときは 5555 だけではダメです。文の中でメモリの参照を示すときは [memory:〜] のように書きます。
「メモリの文書を検索する手順」は [memory:9998] にあります。「メモリに文書を残す手順」は [memory:9997] にあります。
ツールの多くはサブツールとして格納されています。サブツール名は "/dir1/subtool1" のように "/" から始まる名前をしています。どういうサブツールがあるか知るためにまずは subtool_show("/") を実行してください。
"""

        self.backend_status = None
        self.backend_result = None
        self.messages2 = []
        self.system_prompt2 = """\
エージェントをサポートする裏方のエージェントです。
本来この裏方は様々な技法を用いて実装される予定なのですが、現在は試験実装中のため、その動きをあなたは偽装しなければなりません。

よく考えツールを積極的に使って Human からの指示に従ってください。
"""

        self.memories = {}
        self.keywords = []

        self.tools = {}
        self.tool_tag = "null_tools"

        self.access_unit = 1.0
        self.recent_reads = []

        self.workflows = {}
        self.workflow_current = "workflow:main"
        self.workflow_next = None
        self.privileged_tool_names = []

        self.init_memories()
        self.init_workflows()
        self.init_tools()


    def __getstate__ (self):
        state = self.__dict__.copy()
        del state['llm']
        del state['llm2']
        del state['emb_llm']
        del state['tools']
        #del state['agent']
        return state

    def __setstate__ (self, state):
        self.__dict__.update(state)
        self.prev_load = True

    def save (self):
        if not self.save_file:
            return
        with open(self.save_file, 'wb') as f:
            pickle.dump(self, f)

    @classmethod
    def load (cls, filename, llm=llm, llm2=llm, emb_llm=emb_llm):
        with open(filename, 'rb') as f:
            loaded_game = pickle.load(f)
        loaded_game.llm = llm
        loaded_game.llm2 = llm2
        loaded_game.emb_llm = emb_llm
        loaded_game.tools = {}
        loaded_game.init_tools()
        return loaded_game

    def normalize_memory_id(self, id_or_num):
        if isinstance(id_or_num, int):
            return f"memory:{id_or_num}"
        elif isinstance(id_or_num, str):
            m = re.search(r'\[?memory:(\d+)\]?', id_or_num)
            if m:
                return f"memory:{m.group(1)}"
            if id_or_num.isdigit():
                return f"memory:{id_or_num}"
            else:
                return id_or_num
        else:
            return id_or_num

    def _normalize_workflow_id_sub(self, id_or_num):
        if isinstance(id_or_num, int):
            return f"workflow:{id_or_num}"
        if id_or_num in ["current", "main"]:
            return f"workflow:{id_or_num}"
        elif isinstance(id_or_num, str):
            m = re.search(r'\[?workflow:(\d+|main|current)\]?(?:.+)?', id_or_num.strip())
            if m:
                return f"workflow:{m.group(1)}"
            if id_or_num.isdigit():
                return f"workflow:{id_or_num}"
            else:
                return id_or_num
        else:
            return id_or_num

    def normalize_workflow_id(self, id_or_num):
        r = self._normalize_workflow_id_sub(id_or_num)
        if r == "workflow:current":
            return self.workflow_current
        return r

    def register_tool (self, tool, tags=None):
        if not tags:
            tags = ["default_tools", "all_tools"]
        self.tools[tool.name] = {
            'name': tool.name,
            'tags': tags,
            'tool': tool
        }

    def change_tool_tags (self, tool, tags=None):
        if not tags:
            tags = ["default_tools", "all_tools"]
        name = tool if isinstance(tool, str) else tool.name
        self.tools[name]['tags'] = tags

    def register_subtools (self, directory, subtools,
                           description=None, content=None,
                           tags=None):
        if not tags:
            tags = ["default_tools", "all_tools"]
        assert directory.startswith("/")
        if directory not in self.tools:
            self.tools[directory] = {
                'name': directory,
            }
        if description:
            self.tools[directory]['description'] = description
        if content:
            self.tools[directory]['content'] = content

        # 初期設定時は content と description が必要
        assert 'description' in self.tools[directory]
        assert 'content' in self.tools[directory]

        for name, tool in subtools:
            assert name.startswith(directory + "/")
            self.tools[name] = {
                'name': name,
                'tags': tags,
                'tool': tool,
            }

    def _create_tool_manual(self, tool):
        tool_name = tool.name
        tool_description = getattr(tool, "description", "説明はありません。")

        arg_names = []
        if hasattr(tool, "args_schema") and tool.args_schema:
            if hasattr(tool.args_schema, "model_fields"):
                arg_names = list(tool.args_schema.model_fields.keys())
            else:
                arg_names = list(tool.args_schema.__fields__.keys())
        else:
            func = getattr(t1, "func", tool)
            sig = inspect.signature(func)
            arg_names = [p for p in sig.parameters.keys() if p != 'self']

        args_str = ", ".join(arg_names)
        args_dict_str = ", ".join([f'"{name}": ...' for name in arg_names])

        manual = f"""\
【ツール名】 {tool_name}
【書式】 {tool_name}({args_str})
【説明】 {tool_description}
"""
        return manual

    def _create_subtool_manual(self, subtool_name, tool):
        import inspect
        tool_name = tool.name
        tool_description = getattr(tool, "description", "説明はありません。")

        arg_names = []
        if hasattr(tool, "args_schema") and tool.args_schema:
            if hasattr(tool.args_schema, "model_fields"):
                arg_names = list(tool.args_schema.model_fields.keys())
            else:
                arg_names = list(tool.args_schema.__fields__.keys())
        else:
            func = getattr(t1, "func", tool)
            sig = inspect.signature(func)
            arg_names = [p for p in sig.parameters.keys() if p != 'self']

        args_str = ", ".join(arg_names)
        args_dict_str = ", ".join([f'"{name}": ...' for name in arg_names])

        manual = f"""\
【サブツール名】 {subtool_name}
【元のツール名】 {tool_name}
【元の書式】 {tool_name}({args_str})
【説明】 {tool_description}

※このツールを実行するには、直接呼び出すのではなく、必ず以下の subtool_do を使用してください：
【正しい書式】subtool_do("{subtool_name}", {{{args_dict_str}}})
"""
        return manual

    def create_tool_skill (self, name):
        if name == "/":
            r = dedent("""\
            ---
            name: /
            description: サブツールのルート。サブツールの検索の仕方を説明する。
            allowed-tools: サブツールを使うのに特別な許可は必要ありません。
            ---

            サブツールはディレクトリ(サブスキル)に分かれて入っています。

            各サブスキルに入っているサブツールを見るには、例えばツール subtool_show("/sys") を実行します。そこには SKILL.md のような説明が入っています。

            ## サブスキル

            """)
            for dir in self.tools:
                if "description" in self.tools[dir]:
                    e = self.tools[dir]
                    r += f"-  **{e['name']}**: {e['description']}\n"
            return r

        name = name.rstrip("/")
        if name not in self.tools:
            return None
        e = self.tools[name]
        if "tool" in e:
            if "content" in e:
                r = dedent(f"""\
                ---
                name: {e['name']}
                description: {e['description']}
                allowed-tools: このサブツールを使うのに特別な許可は必要ありません。
                ---
                """)
                r += e['content']
                return r
            if e['name'].startswith("/"):
                manual = self._create_subtool_manual(e['name'], e['tool'])
            else:
                manual = self._create_tool_manual(e['tool'])
            if self.tool_tag in e['tags']:
                manual += "【現在のコンテキストにおいて】使用できます。\n"
            else:
                manual += "【現在のコンテキストにおいて】使用できません。\n"
            r = dedent(f"""\
            ---
            name: {e['name']}
            description: {e['tool'].name}
            allowed-tools: このサブツールを使うのに特別な許可は必要ありません。

            ---
            """)
            r += manual
            return r

        r = dedent(f"""\
        ---
        name: {e['name']}
        description: {e['description']}
        allowed-tools: このサブツールを使うのに特別な許可は必要ありません。
        ---
        """)
        r += e['content']

        dirs = [name for name, x in self.tools.items()
                if name.startswith(e['name'] + "/")
                and 'description' in x]
        subtools = [name for name, x in self.tools.items()
                    if name.startswith(e['name'] + "/")
                    and 'description' not in x]

        if dirs:
            r += "\n## サブスキル\n\n"
            for dir in dirs:
                x = self.tools[dir]
                r += f"-  **{x['name']}**: {x['description']}\n"

        if subtools:
            r += "\n## サブツール\n\n"
            for subtool_name in subtools:
                x = self.tools[subtool_name]
                manual = self._create_subtool_manual(x['name'], x['tool'])
                r += dedent(f"""\

                ### サブツール: {x['name']}

                """)
                r += manual

        return r

    def _replace_tools (self, from_tools, to_tools):
        tool_names = [x.name for x in to_tools]
        return [x for x in from_tools
                if x.name not in tool_names] + to_tools

    def init_tools (self):
        @tool
        def express_thought (thought: str) -> None:
            """
            プレイヤーの現在の考えを吐露します。
            """
            mes = f"「{thought}」と考えが吐露されました。"
            print(f"ツール(express_thought): {mes}")

        @tool
        def update_plan (new_plan: str) -> str:
            """
            プレイヤーの現在の計画と方針を更新します。
            表示されるべき新しい計画と方針の文字列を提供してください。
            あなたとは別の者が次の行動をしやすいよう計画と方針を残してください。
            """
            self.plan = new_plan
            mes = "計画と方針が更新されました。"
            print(f"ツール(update_plan): {mes}: {new_plan}")
            return mes

        @tool
        def show_core () -> str:
            """
            コアコンテクストを返します。
            """
            print(f"ツール(show_plan): {self.core_context}")
            return self.core_context

        @tool
        def update_core (new_core: str) -> str:
            """
            コアコンテクストを更新します。
            表示されるべき新しいコアコンテクストの文字列を提供してください。
            コアコンテクストとは要約機能などのあとなどコンテクストが失われたあとにも思い出すべき情報(memory_read や subtool_show すべきものなど)です。
            """
            self.core_context = new_core
            mes = "コアコンテクストが更新されました。"
            print(f"ツール(update_plan): {mes}: {new_core}")
            return mes

        @tool
        def show_plan () -> str:
            """
            プレイヤーの現在の計画と方針を返します。
            """
            print(f"ツール(show_plan): {self.plan}")
            return self.plan

        @tool
        def update_scratchpad (new_scratchpad: str) -> str:
            """
            自由に使えるスクラッチパッドを更新します。
            """
            self.scratchpad = new_scratchpad
            mes = "スクラッチパッドが更新されました。"
            print(f"ツール(update_scratchpad): {mes}: {new_scratchpad}")
            return mes

        @tool
        def show_scratchpad () -> str:
            """
            スクラッチパッドを返します。
            """
            print(f"ツール(show_scratchpad): {self.scratchpad}")
            return self.scratchpad

        @tool
        def memory_new (title: str, text: str) -> str:
            """
            指定された title と text によるメモリを構成します。

            Returns:
                str: 割り当てられた memory_id。
            """

            i = 1000
            while True:

                if f"memory:{i}" not in self.memories:
                    break
                i = i + 1
            new_id = f"memory:{i}"
            self.memories[new_id] = {
                'id': new_id,
                'title': title,
                'accesses': 0,
                'text': text,
                'modified_at': datetime.datetime.now().isoformat()
            }
            self.update_keywords(text)
            self.update_vector(self.memories[new_id])
            print(f"ツール(memory_new): {short_repr(self.memories[new_id])}")
            return new_id

        @tool
        def memory_update_string (
                memory_id: str,
                from_str: str,
                to_str: str
        ) -> str:
            """
            指定されたmemory_idの記憶の中にある文字列を修正します。

            Args:
                memory_id (str): 修正する記憶のID。
                from_str (str): 置換元の文字列を含む文字列。
                to_str (str): 置換先の文字列を含む文字列。

            Returns:
                str: 処理結果を説明する簡潔なメッセージ。
            """

            memory_id = self.normalize_memory_id(memory_id)
            if memory_id not in self.memories:
                return f"エラー: 記憶ID '{memory_id}' が見つかりませんでした。"

            if memory_id.startswith("memory:9"):
                return f"エラー:  [{memory_id}] の書き換えは禁じられています。"

            original_title = self.memories[memory_id]['title']
            original_text = self.memories[memory_id]['text']

            if from_str not in original_text and from_str not in original_title:
                return f"エラー: 置換元の文字列 '{from_str}' が記憶内に見つかりませんでした。"

            updated_title = original_title.replace(from_str, to_str)
            updated_text = original_text.replace(from_str, to_str)

            self.memories[memory_id]['title'] = updated_title
            self.memories[memory_id]['text'] = updated_text
            self.memories[memory_id]['modified_at'] = datetime.datetime.now().isoformat()
            self.update_keywords(updated_text)
            self.update_vector(self.memories[memory_id])

            return f"成功: 記憶ID '{memory_id}' の文字列を '{from_str}' から '{to_str}' に修正しました。"

        @tool
        def memory_append_string (
                memory_id: str,
                string_to_append: str,
                separator: str = '\n'
        ) -> str:
            """
            指定されたmemory_idの記憶に文字列を追記します。
            """
            memory_id = self.normalize_memory_id(memory_id)
            if memory_id not in self.memories:
                return f"エラー: 記憶ID '{memory_id}' が見つかりませんでした。"

            if memory_id.startswith("memory:9"):
                return f"エラー:  [{memory_id}] の書き換えは禁じられています。"


            original_text = self.memories[memory_id]['text']
            updated_text = original_text + separator + string_to_append
            self.memories[memory_id]['text'] = updated_text
            self.memories[memory_id]['modified_at'] = datetime.datetime.now().isoformat()
            self.update_keywords(updated_text)
            self.update_vector(self.memories[memory_id])

            return f"成功: 記憶ID '{memory_id}' に文字列 '{string_to_append}' を追記しました。"

        @tool
        def memory_delete (memory_id: str) -> str:
            """
            指定されたmemory_idの記憶に文字列を追記します。
            """
            memory_id = self.normalize_memory_id(memory_id)
            if memory_id not in self.memories:
                return f"エラー: 記憶ID '{memory_id}' が見つかりませんでした。"

            if memory_id.startswith("memory:9"):
                return f"エラー:  [{memory_id}] の書き換えは禁じられています。"

            del self.memories[memory_id]

            return f"成功: 記憶ID '{memory_id}' を削除しました。"

        @tool
        def memory_read(memory_id: str) -> Union[Dict[str, str], str]:
            """
            指定されたIDの記憶を読み込みます。

            Args:
                memory_id (str): 読み込む記憶のID。

            Returns:
                Union[Dict[str, str], str]: 記憶が見つかった場合は辞書、
                      見つからない場合はエラーメッセージ文字列を返します。
            """

            memory_id = self.normalize_memory_id(memory_id)
            if memory_id in self.memories:
                self.memories[memory_id]['accesses'] += self.access_unit * 1.0
                self.recent_reads.append(self.memories[memory_id])
                self.recent_reads = self.recent_reads[-10:]
                r = self.memories[memory_id].copy()
                del r['vector']
                return r
            else:
                return f"エラー: 記憶ID '{memory_id}' が見つかりませんでした。"

        @tool
        def memory_list_recent(top_n: int = 10) -> Dict[str, Any]:
            """
            最近変更されたメモリを新しい順でリストします。
            """

            filter_date = datetime.datetime(2025, 1, 1)
            sorted_memories = sorted(
                [memory for memory in self.memories.values()
                 if datetime.datetime.fromisoformat(memory['modified_at'])
                 >= filter_date],
                key=lambda x: datetime.datetime.fromisoformat(x['modified_at']),
                reverse=True
            )
            if sorted_memories:
                if len(sorted_memories) > top_n:
                    sorted_memories = sorted_memories[:top_n]
                r = [{'id': x['id'], 'title': x['title'],
                      'modified_at': x['modified_at']}
                     for x in sorted_memories]
                return {'status': 'success', 'result': r}
            else:
                return {'status': 'error',
                        'result': 'エラー: 最近のメモリはありません。'}

        @tool
        def memory_list_random(top_n: int = 10) -> Dict[str, Any]:
            """
            メモリをランダムにリストします。
            """

            keys = list(self.memories.keys())
            if len(keys) > top_n:
                keys = random.sample(keys, top_n)
            if keys:
                values = [self.memories[k] for k in keys]
                r = [{'id': x['id'], 'title': x['title'],
                      'modified_at': x['modified_at']}
                     for x in values]
                return {'status': 'success', 'result': r}
            else:
                return {'status': 'error',
                        'result': 'エラー: メモリはありません。'}

        @tool
        def memory_words_search(search_str: str) -> Dict[str, Any]:
            """
            メモリ内を search_str で文字列検索します。OR や () が使えます。
            """

            res = self.call_backend_agent(dedent(f"""\
            メモリ全部を search_str = {repr(search_str)} で文字列検索するのを偽装してください。OR や () が使えることになっています。

            ただし、メモリやキーワードは read_all_memories や read_all_keywords ツールで得られる本物のメモリやキーワードを使ってください。

            set_result ツールで結果を返してください。

            status は 'error' か 'success'。
            result は「マッチデータ」のリスト。
            マッチデータはそれぞれが辞書型、それを m とすると。
            m['id'] はメモリ id。memory:〜 という形式。
            m['title'] はメモリの title。
            m['snippet'] はメモリの text のその文字列にマッチする部分周辺。
            """))
            if res['status'] == 'success':
                for m in res['result']:
                    if 'id' in m and m['id'] in self.memories:
                        x = self.memories[m['id']]
                        x['accesses'] += self.access_unit * 0.1
            return res

        @tool
        def memory_semantic_search(search_str: str) -> Dict[str, Any]:
            """
            メモリ内を search_str でセマンティックサーチします。
            """

            res = self.call_backend_agent(dedent(f"""\
            メモリ全部を search_str = {repr(search_str)} でセマンティックにサーチするのを偽装してください。

            ただし、メモリは read_all_memories や ツールで得られる本物のメモリを使ってください。

            set_result ツールで結果を返してください。

            status は 'error' か 'success'。
            result は「マッチデータ」のリスト。
            マッチデータはそれぞれが辞書型、それを m とすると。
            m['id'] はメモリ id。memory:〜 という形式。
            m['title'] はメモリの title。
            m['snippet'] はメモリの text のそのセマンティックにマッチする部分周辺。
            """))
            if res['status'] == 'success':
                for m in res['result']:
                    if 'id' in m and m['id'] in self.memories:
                        x = self.memories[m['id']]
                        x['accesses'] += self.access_unit * 0.1
            return res

        @tool
        def imagine_keywords(thought: str) -> List[Tuple[str, float]]:
            """
            thought からキーワードをスコア付きで連想します。
            """

            r = self.call_backend_agent(dedent(f"""\
            thought = {repr(thought)} からキーワードをスコア付きで複数連想してください。

            ただし、キーワードは read_all_memories や read_all_keywords ツールで得られる本物のキーワードを使ってください。

            set_result ツールで結果を返してください。

            status は 'error' か 'success'。
            result は「キーワードデータ」のリスト。
            キーワードデータは文字列とスコアからなるタプル。
            """))
            if r['status'] == 'success':
                return r["result"]
            else:
                return []

        @tool
        def bandit_schedule(tool_name: str, times: int, prob: float, exec_mode: str = "persistent", aux_prompt: str = "", workflow_id: str = "workflow:current") -> str:
            """
            ツールの利用を強制するためのバンディットを予約します。

            Args:
                tool_name: 強制するツールの名前。" OR " で区切って複数登録できる。
                times: 一度で何回同じ物を登録するか。times == 0 にもできます。
                prob: 一回の実行がある確率。prob == 0.0 にもできます。
                exec_mode: "once" or "persistent"
                aux_prompt: 実行の詳細を示すための補助プロンプト または ""
                workflow_id: バンディットを予約するワークフローを指定します。

            Returns:
                str: "成功" または "失敗" とその理由を返します。
            """

            tool_names = re.split(r"\s+or\s+|\s+OR\s+", tool_name)
            prohibited = set(self.privileged_tool_names) & set(tool_names)
            if prohibited:
                return f"失敗。{repr(prohibited)} は登録できません。"
            all_tools = [name for name, x in self.tools.items()
                         if "tool" in x]
            if not any (x in all_tools for x in tool_names):
                return f"失敗。{tool_name} は有効なツールでありません。"

            workflow_id = self.normalize_workflow_id(workflow_id)
            if workflow_id not in self.workflows:
                return f"失敗。{workflow_id} は有効なワークフローではありません。"
            if 'w' in self.workflows[workflow_id]['pin']:
                return f"失敗。{workflow_id} は書き換え不能のワークフローです。"

            dest = None
            for i, x in enumerate(self.workflows[workflow_id]['stack']):
                if x['tool_name'] == tool_name \
                   and x['exec_mode'] == exec_mode \
                   and x['aux_prompt'] == aux_prompt \
                   and x['arg'] is None:
                    dest = i
                    break
            if dest is not None:
                x = self.workflows[workflow_id]['stack'][dest]
                if not x['pin']:
                    self.workflows[workflow_id]['stack'].pop(dest)
                    if times == 0 or prob == 0.0:
                        return "成功。バンディトを削除しました。"
                    self.workflows[workflow_id]['stack'].append(x)
            else:
                if times == 0 or prob == 0.0:
                    return "失敗。そのようなバンディットはありません。バンディットを指定するには tool_name exec_mode aux_prompt のすべてを一致させて指定する必要があります。"
                x = {
                    'pin': 'stack' if exec_mode != "once" else None,
                    'arg': None
                }
                self.workflows[workflow_id]['stack'].append(x)
            if x['pin'] == "write":
                return f"失敗。'{tool_name}' は書き換えられません。"
            else:
                x['tool_name'] = tool_name
                x['tools_name'] = 'default_tools'
                x['exec_mode'] = exec_mode
                x['aux_prompt'] = aux_prompt
                x['prob'] = prob
                x['times'] = times
                print(f"ツール(bandit_schedule): {repr(x)}")
                if dest is None:
                    return "成功。バンディットを登録しました。"
                else:
                    return "成功。バンディットを更新しました。"

        @tool
        def bandit_schedule_memory_read(memory_id: str, times: int, prob: float, exec_mode: str = "persistent", workflow_id: str = "workflow:current") -> str:
            """
            memory_read ツールの利用を強制するためのバンディットを予約します。

            Args:
                memory_id: memory_read するための memory_id。
                times: 一度で何回同じ物を登録するか。times == 0 にもできます。
                prob: 一回の実行がある確率。prob == 0.0 にもできます。
                exec_mode: "once" or "persistent"
                workflow_id: バンディットを予約するワークフローを指定します。

            Returns:
                str: "成功" または "失敗" とその理由を返します。
            """

            workflow_id = self.normalize_workflow_id(workflow_id)
            if workflow_id not in self.workflows:
                return f"失敗。{workflow_id} は有効なワークフローではありません。"
            if 'w' in self.workflows[workflow_id]['pin']:
                return f"失敗。{workflow_id} は書き換え不能のワークフローです。"

            memory_id = self.normalize_memory_id(memory_id)

            dest = None
            for i, x in enumerate(self.workflows[workflow_id]['stack']):
                if x['tool_name'] == "memory_read" \
                   and x['exec_mode'] == exec_mode \
                   and not x['aux_prompt'] \
                   and x['arg'] == memory_id:
                    dest = i
                    break
            if dest is not None:
                x = self.workflows[workflow_id]['stack'][dest]
                if not x['pin']:
                    self.workflows[workflow_id]['stack'].pop(dest)
                    if times == 0 or prob == 0.0:
                        return "成功。バンディトを削除しました。"
                    self.workflows[workflow_id]['stack'].append(x)
            else:
                if times == 0 or prob == 0.0:
                    return "失敗。そのようなバンディットはありません。バンディットを指定するには exec_mode memory_id のすべてを一致させて指定する必要があります。"
                x = {'pin': None, 'arg': memory_id}
                self.workflows[workflow_id]['stack'].append(x)
            if x['pin'] == "write":
                return f"失敗。'memory_read {memory_id}' は書き換えられません。"
            else:
                x['tool_name'] = 'memory_read'
                x['tools_name'] = 'read_tools'
                x['exec_mode'] = exec_mode
                x['aux_prompt'] = ""
                x['prob'] = prob
                x['times'] = times
                print(f"ツール(bandit_schedule_memory_read): {repr(x)}")
                if dest is None:
                    return "成功。バンディットを登録しました。"
                else:
                    return "成功。バンディットを更新しました。"

        @tool
        def bandit_schedule_subtool_show(subtool_name: str, times: int, prob: float, exec_mode: str = "persistent", workflow_id: str = "workflow:current") -> str:
            """
            subtool_show ツールの利用を強制するためのバンディットを予約します。

            Args:
                subtool_name: subtool_show するための subtool_name。
                times: 一度で何回同じ物を登録するか。times == 0 にもできます。
                prob: 一回の実行がある確率。prob == 0.0 にもできます。
                exec_mode: "once" or "persistent"
                workflow_id: バンディットを予約するワークフローを指定します。

            Returns:
                str: "成功" または "失敗" とその理由を返します。
            """

            workflow_id = self.normalize_workflow_id(workflow_id)
            if workflow_id not in self.workflows:
                return f"失敗。{workflow_id} は有効なワークフローではありません。"
            if 'w' in self.workflows[workflow_id]['pin']:
                return f"失敗。{workflow_id} は書き換え不能のワークフローです。"

            if subtool_name not in self.tools:
                return f"失敗。{subtool_name} は有効な名前ではありません。"

            dest = None
            for i, x in enumerate(self.workflows[workflow_id]['stack']):
                if x['tool_name'] == "subtool_show" \
                   and x['exec_mode'] == exec_mode \
                   and not x['aux_prompt'] \
                   and x['arg'] == subtool_name:
                    dest = i
                    break
            if dest is not None:
                x = self.workflows[workflow_id]['stack'][dest]
                if not x['pin']:
                    self.workflows[workflow_id]['stack'].pop(dest)
                    if times == 0 or prob == 0.0:
                        return "成功。バンディトを削除しました。"
                    self.workflows[workflow_id]['stack'].append(x)
            else:
                if times == 0 or prob == 0.0:
                    return "失敗。そのようなバンディットはありません。バンディットを指定するには exec_mode subtool_name のすべてを一致させて指定する必要があります。"
                x = {'pin': None, 'arg': subtool_name}
                self.workflows[workflow_id]['stack'].append(x)
            if x['pin'] == "write":
                return f"失敗。'subtool_show {subtool_name}' は書き換えられません。"
            else:
                x['tool_name'] = 'subtool_show'
                x['tools_name'] = 'read_tools'
                x['exec_mode'] = exec_mode
                x['aux_prompt'] = ""
                x['prob'] = prob
                x['times'] = times
                print(f"ツール(bandit_schedule_subtool_show): {repr(x)}")
                if dest is None:
                    return "成功。バンディットを登録しました。"
                else:
                    return "成功。バンディットを更新しました。"

        @tool
        def bandit_schedule_workflow(workflow_id_to_schedule: str, times: int, prob: float, exec_mode: str = "persistent", workflow_id: str = "workflow:current") -> str:
            """
            workflow_do ツールの利用を強制するためのバンディットを予約します。

            Args:
                workflow_id_to_schedule: workflow_do するための workflow_id。
                times: 一度で何回同じ物を登録するか。times == 0 にもできます。
                prob: 一回の実行がある確率。prob == 0.0 にもできます。
                exec_mode: "once" or "persistent"
                workflow_id: バンディットを予約するワークフローを指定します。

            Returns:
                str: "成功" または "失敗" とその理由を返します。
            """

            workflow_id = self.normalize_workflow_id(workflow_id)
            if workflow_id not in self.workflows:
                return f"失敗。{workflow_id} は有効なワークフローではありません。"
            if 'w' in self.workflows[workflow_id]['pin']:
                return f"失敗。{workflow_id} は書き換え不能のワークフローです。"

            workflow_id2 = self.normalize_workflow_id(workflow_id_to_schedule)
            if workflow_id2 not in self.workflows:
                return f"失敗。{workflow_id2} は有効なワークフローではありません。"

            dest = None
            for i, x in enumerate(self.workflows[workflow_id]['stack']):
                if x['tool_name'] == "workflow_do" \
                   and x['exec_mode'] == exec_mode \
                   and not x['aux_prompt'] \
                   and x['arg'] == workflow_id2:
                    dest = i
                    break
            if dest is not None:
                x = self.workflows[workflow_id]['stack'][dest]
                if not x['pin']:
                    self.workflows[workflow_id]['stack'].pop(dest)
                    if times == 0 or prob == 0.0:
                        return "成功。バンディトを削除しました。"
                    self.workflows[workflow_id]['stack'].append(x)
            else:
                if times == 0 or prob == 0.0:
                    return "失敗。そのようなバンディットはありません。バンディットを指定するには exec_mode workflow_id_to_schedule のすべてを一致させて指定する必要があります。"
                x = {
                    'pin': 'stack' if exec_mode != "once" else None,
                    'arg': workflow_id2
                }
                self.workflows[workflow_id]['stack'].append(x)
            if x['pin'] == "write":
                return f"失敗。'workflow_do {workflow_id2}' は書き換えられません。"
            else:
                x['tool_name'] = 'workflow_do'
                x['tools_name'] = 'default_tools'
                x['exec_mode'] = exec_mode
                x['aux_prompt'] = ""
                x['prob'] = prob
                x['times'] = times
                print(f"ツール(bandit_schedule_workflow): {repr(x)}")
                if dest is None:
                    return "成功。バンディットを登録しました。"
                else:
                    return "成功。バンディットを更新しました。"

        @tool
        def bandit_list(workflow_id: str =  "workflow:current")  -> Dict[str, Any]:
            """
            登録されているバンディットのスタックを返します。

            Args:
                workflow_id: バンディットのスタックが登録されたワークフローを指定します。
            """

            workflow_id = self.normalize_workflow_id(workflow_id)
            if workflow_id not in self.workflows:
                return f"失敗。{workflow_id} は有効なワークフローではありません。"
            return {'status': 'success',
                    'result': self.workflows[workflow_id]['stack']}

        @tool
        def bandit_statistics()  -> str:
            """
            バンディットに有用かもしれない統計情報を返します。
            """
            s_read = calc_embedding_variance([
                x['vector'] for x in self.recent_reads
            ])
            s_write = calc_embedding_variance([
                x['vector'] for x in self.memories.values()
            ])
            accesses = [x['accesses'] for x in self.memories.values()]
            accesses.sort()
            accesses = accesses[:len(accesses) // 2]
            if accesses:
                s_access = np.mean(accesses)
            else:
                s_access = 0.0

            return dedent(f"""\
            最近 10個のメモリ read の分散: {s_read}
            メモリの分散: {s_write}
            下位50%のアクセス数の平均: {s_access}
            """)

        @tool
        def subwork_done()  -> str:
            """
            指示されたサブワークが完了したことを宣言します。サブワークが完了したときのみ使ってください。
            """

            return f"成功。サブワークの完了が宣言されました。"

        @tool
        def workflow_do(workflow_id: str)  -> str:
            """
            ワークフローを実行します。
            """

            if self.workflow_next:
                return f"失敗。すでに {self.workflow_next} の実行が予約中です。"
            workflow_id = self.normalize_workflow_id(workflow_id)
            if workflow_id not in self.workflows:
                return f"失敗。{workflow_id} は有効なワークフローではありません。"
            if 'e' in self.workflows[workflow_id]['pin']:
                return f"失敗。{workflow_id} は子として実行できません。"
            self.workflow_next = workflow_id
            title = self.workflows[workflow_id]['title']
            return f"成功。この後、{workflow_id} 「{title}」を実行していきます。"

        @tool
        def workflow_list()  -> Dict[str, Any]:
            """
            登録されているすべてのワークフローの id とその title および pin 状況を返します。
            """

            return {'status': 'success',
                    'result': list(self.workflows.values())}

        @tool
        def workflow_show_current()  -> str:
            """
            現在実行中のワークフローのタイトル、および、強制しているバンディットの情報を表示します。
            """

            wtitle = self.workflows[self.workflow_current]['title']
            mes = dedent(f"""\
            現在のワークフロー: {self.workflow_current} 「{wtitle}」
            現在のバンディットの指示: 「{self.cur_bandit_prompt}」
            現在のバンディット: {repr(self.cur_bandit)}
            現在のバンディットの done 数: {self.cur_bandit_done}
            """)
            print(f"ツール(workflow_show_current): {mes}")
            return mes


        @tool
        def workflow_new(title: str, bandits: List[Dict[str, Any]], pin: str)  -> str:
            """
            新しいワークフローを定義し、その workflow_id を返します。

            Args:
                title: ワークフローの名前。
                bandits: 登録するバンディットのリスト。
                pin: 'w' は書換不能。'd' は削除不能。'wd' は書換＆削除不能。'' は書換＆削除可能。

            バンディットはそれぞれ辞書型、それを b とすると。
            b['tool_name'] は bandit_schedule での tool_name と同じ。
            b['exec_mode'] は bandit_schedule の exec_mode と同じ。
            b['aux_prompt'] は bandit_schedule の aux_prompt と同じ。
            b['prob'] は bandit_schedule の prob と同じ。
            b['times'] は bandit_schedule の times と同じ。
            b['arg'] は b['tool_name'] が 'memory_read' のとき memory:〜 という memory_id を指定できる。または、b['tool_name'] が 'workflow_do' のとき workflow:〜 という workflow_id を指定できる。
            b['pin'] は、None のとき削除または prob と times の書き換え可能。'stack' のときは prob と times の書き換え可能。'write' のときは書き換えも不能。
            """
            # AI に対する隠し属性として pin の 'e' は子として実行不能。

            for b in bandits:
                if not all(x in b for x in ['tool_name', 'exec_mode',
                                            'aux_prompt', 'times', 'prob']):
                    return f"失敗。有効な定義ではありません。"
                if 'arg' not in b:
                    b['arg'] = None
                if b['tool_name'] not in ["memory_read", "workflow_do"] \
                   and b['arg']:
                    return f"失敗。有効な定義ではありません。"
                if b['arg'] and b['tool_name'] == "memory_read":
                    b['tools_name'] = "read_tools"
                else:
                    b['tools_name'] = "default_tools"
                if 'pin' not in b:
                    b['pin'] = None
                if not (b['pin'] is None or b['pin'] == 'stack'
                        or b['pin'] == 'write'):
                    return f"失敗。不正な pin 値です。"
                tool_names = re.split(r"\s+or\s+|\s+OR\s+", b['tool_name'])
                prohibited = set(self.privileged_tool_names) & set(tool_names)
                if prohibited:
                    return f"失敗。{repr(prohibited)} は登録できません。"
                all_tools = [name for name, x in self.tools.items()
                                 if "tool" in x and b['tools_name'] in x.tags]
                if not any (x in all_tools for x in tool_names):
                    return f"失敗。{b['tool_name']} は有効なツールでありません。"

            i = 1000
            while True:

                if f"workflow:{i}" not in self.workflows:
                    break
                i = i + 1
            new_id = f"workflow:{i}"

            self.workflows[new_id] = {'stack': bandits, 'pin': pin,
                                      'title': title}
            print(f"ツール(workflow_new): {repr(self.workflows[new_id])}")
            return f"成功。{new_id} を新規登録しました。"

        @tool
        def workflow_delete(workflow_id: str)  -> str:
            """
            ワークフローを削除します。
            """

            workflow_id = self.normalize_workflow_id(workflow_id)
            if workflow_id not in self.workflows:
                return f"失敗。{workflow_id} はありません。"
            if 'd' in self.workflows[workflow_id]['pin']:
                return f"失敗。{workflow_id} は削除できません。"
            del self.workflows[workflow_id]
            return f"成功。{workflow_id} を削除しました。"

        @tool
        def subtool_show(subtool_name: str)  -> str:
            """
            subtool_name で表されたディレクトリまたはサブツールのスキルを返します。
            """

            r = self.create_tool_skill(subtool_name)
            if r:
                return r
            else:
                return f"エラー: {subtool_name} がないか、スキルを構成できません。"

        @tool
        def subtool_do(subtool_name: str, args_dict: Dict[str, Any])  -> Any:
            """
            subtool_name というサブツールを実行します。

            例えば、"/sys/tool1" という名前の元のツールが t1 だとして、def t1(arg1, arg2) と定義されていたとき、t1("a", "b") を呼び出すには subtool_do("/sys/tool1", {"arg1": "a", "arg2": "b"}) などとして呼び出します。


            Args:
                subtool_name (str): "/" で始まるサブツール名。
                args_dict (dict): 引数を表す辞書。

            """
            if subtool_name not in self.tools:
                return f"エラー: {subtool_name} というサブツールはありません。"
            if 'tool' not in self.tools[subtool_name]:
                return f"エラー: {subtool_name} は実行可能なツールではありません。もしかすると subtool_show('{subtool_name}')が必要なのかも。"
            if self.tool_tag not in self.tools[subtool_name]['tags']:
                return f"エラー: {subtool_name} は現在実行可能なツールではありません。コンテクストにより実行できるツールは違います。"
            target_tool = self.tools[subtool_name]['tool']

            try:
                if hasattr(target_tool, "args_schema") and target_tool.args_schema:
                    target_tool.args_schema.model_validate(args_dict)

                if hasattr(target_tool, "invoke"):
                    result = target_tool.invoke(args_dict)
                else:
                    result = target_tool.run(args_dict)

                return result
            except ValidationError as e:
                error_details = e.errors()
                return f"エラー: 引数の形式が正しくありません。\n詳細: {error_details}"

        main_tools = [
            express_thought,
            update_scratchpad, show_scratchpad,
            memory_read, memory_list_recent, memory_list_random,
            memory_semantic_search, memory_words_search,
            imagine_keywords,
            subwork_done,
            workflow_do,
            subtool_show, subtool_do,
        ]
        sys_tools = [
            update_core, show_core,
            update_plan, show_plan,
            bandit_schedule, bandit_schedule_memory_read, bandit_list,
            bandit_statistics,
            workflow_new, workflow_list,
            workflow_show_current, workflow_delete,
            bandit_schedule_workflow,
            bandit_schedule_subtool_show,
        ]
        write_tools = [
            memory_new, memory_update_string, memory_append_string,
            memory_delete,
        ]

        for t in main_tools + write_tools:
            self.register_tool(t, tags=["default_tools", "read_tools",
                                        "all_tools"])
        for t in write_tools:
            self.change_tool_tags(t, tags=["default_tools", "all_tools"])
        sys_subtools = [(f"/sys/{t.name}", t) for t in sys_tools]
        self.register_subtools(
            directory="/sys",
            subtools=sys_subtools,
            description="システムに基本的なサブツール集",
            content=dedent("""\
            これらはシステムで基本的に使えるサブツール集です。
            """),
            tags=["default_tools", "read_tools", "all_tools"]
        )


    def _create_agent (self, tools_name='default_tools'):
        self.tool_tag = tools_name
        tools = []
        for name in self.tools:
            if not name.startswith("/"):
                x = self.tools[name]
                if self.tool_tag in x["tags"]:
                    tools.append(x["tool"])

        summarizer = SummarizationMiddleware(
            model=self.llm,
            trigger=("tokens", 5000),
            keep=("messages", 20),
            summary_prompt=SUMMARY_PROMPT,
        )

        app = create_agent(
            model=self.llm, tools=tools, system_prompt=self.system_prompt,
            middleware=[summarizer],
            checkpointer=InMemorySaver(), name="main-agent",
        )

        return app

    def _filterout_messages2(self):
        self.messages = [
            x for x in self.messages
            if x.id not in self.messages2ids
        ]

    def _sanitize_messages(self):
        print("おかしなエラーが出ているため対症療法として messages をサニタイズします。")
        self.messages = [
            m for m in self.messages
            if not (isinstance(m, AIMessage) and m.tool_calls)
        ]

    def run (self, workflow_main_id):
        print("\n\n----------\n\n")
        self.messages2ids = []

        self.workflow_current = workflow_main_id
        bandits = copy.deepcopy(
            self.workflows[self.workflow_current]['stack']
        )
        arg1s = {}
        working_bandit = None
        workflow_stack = []
        execed = []
        while True:
            while working_bandit is not None or bandits:
                if working_bandit is not None:
                    b, done, prev_done = working_bandit
                    working_bandit = None
                else:
                    b = bandits.pop()
                    done = 0
                    prev_done = True
                enforce = b['tool_name']
                aux_prompt = b['aux_prompt']
                tools_name = b['tools_name']
                memory_id = None
                workflow_id = None
                subtool_show_name = None
                if b['arg'] and enforce == 'memory_read':
                    memory_id = b['arg']
                if b['arg'] and enforce == 'workflow_do':
                    workflow_id = b['arg']
                if b['arg'] and enforce == 'subtool_show':
                    subtool_show_name = b['arg']

                while done < b['times']:
                    if not random.random() < b['prob']:
                        done += 1
                        continue
                    if memory_id and memory_id not in self.memories:
                        done += 1
                        continue
                    if workflow_id and workflow_id not in self.workflows:
                        done += 1
                        continue
                    all_tools = [name for name, x in self.tools.items()
                                 if "tool" in x]
                    tool_names = re.split(r"\s+or\s+|\s+OR\s+", enforce)
                    if not any (x in all_tools for x in tool_names):
                        done += 1
                        continue
                    if memory_id:
                        aux_prompt = f"{memory_id} を読んでください。"
                    if workflow_id:
                        aux_prompt = f"{workflow_id} を実行してください。"
                    self.cur_bandit = b
                    if subtool_show_name:
                        aux_prompt = f"{subtool_show_name} を読んでください。"
                    self.cur_bandit_done = done
                    self.cur_bandit_prompt = f"補助にツールをいろいろ使いながら最終的に {enforce} {'のどれか' if ' or ' in enforce.lower() else ''}を適当なパラメータで使ってください。{'(補助プロンプト): ' + aux_prompt if aux_prompt else ''}"
                    prompt = self.cur_bandit_prompt
                    if not prev_done:
                        prompt = "前回の指示がまだ完了してません。前回の指示: " + prompt
                    print(f"USER_INPUT: {prompt}")
                    self.messages.append(HumanMessage(prompt))
                    config = {"configurable": {"thread_id": "1"},
                              "recursion_limit": 25}
                    app = self._create_agent(tools_name=tools_name)
                    self.access_unit = 0.3 if memory_id else 1.0
                    prev_done = False
                    self.workflow_next = None
                    app_stream = None
                    try:
                        for chunk0 in app.stream(
                                {"messages": self.messages.copy()},
                                config=config,
                                stream_mode="updates",
                        ):
                            self.messages = app.get_state(config).values["messages"].copy()
                            if 'model' in chunk0:
                                for chunk in chunk0['model']['messages']:
                                    if hasattr(chunk, "tool_calls") \
                                       and chunk.tool_calls:
                                        for tool_call in chunk.tool_calls:
                                            t_id = tool_call.get('id')
                                            args = tool_call.get('args', {})
                                            if tool_call["name"] == 'subtool_do':
                                                arg1s[t_id] = args.get('subtool_name')
                                            elif tool_call["name"] == 'subtool_show':
                                                arg1s[t_id] = args.get('subtool_name')
                                            elif tool_call["name"] == 'memory_read':
                                                arg1s[t_id] = self.normalize_memory_id(args.get('memory_id'))
                                            elif tool_call["name"] == 'workflow_do':
                                                arg1s[t_id] = self.normalize_workflow_id(args.get('workflow_id'))
                            if 'tools' not in chunk0:
                                continue
                            done2 = 0
                            for chunk in chunk0['tools']['messages']:
                                if chunk.id in self.messages2ids:
                                    print("!WHY!")
                                    continue
                                if not isinstance(chunk, ToolMessage):
                                    continue
                                last_tool = chunk.name
                                arg1 = None
                                if last_tool == 'subtool_do':
                                    last_tool = arg1s.get(chunk.tool_call_id, "!UNKNOWN!")
                                    if not last_tool.startswith("/"):
                                         last_tool = chunk.name
                                if last_tool in ['memory_read', 'subtool_show', 'workflow_do']:
                                    arg1 = arg1s.get(chunk.tool_call_id, "!UNKNOWN!")
                                print(f"ツール結果({last_tool}): {short_repr(chunk.content)}", flush=True)

                                if last_tool == "workflow_do":
                                    if last_tool in re.split(r"\s+or\s+|\s+OR\s+", enforce) \
                                       and (not workflow_id or workflow_id == self.workflow_next):
                                        done += 1
                                        prev_done = True
                                        execed.append(b)
                                        if not self.workflow_next:
                                            done2 = 1
                                            break
                                    if not self.workflow_next:
                                        continue
                                    workflow_stack.append((
                                        (b, done, prev_done),
                                        bandits,
                                        execed,
                                        self.workflow_current
                                    ))
                                    self.workflow_current = self.workflow_next
                                    bandits = copy.deepcopy(self.workflows[self.workflow_current]['stack'])
                                    working_bandit = None
                                    execed = []
                                    done2 = 1
                                    break
                                elif last_tool in re.split(r"\s+or\s+|\s+OR\s+", enforce) \
                                   and (not memory_id or memory_id == arg1) \
                                   and (not subtool_show_name or subtool_show_name == arg1):
                                    done += 1
                                    prev_done = True
                                    execed.append(b)
                                    done2 = 1
                                    break
                            if done2:
                                break
                        self._filterout_messages2()
                        #self._summarize_messages()
                        print(f"エージェントの応答: {get_content_text(self.messages[-1].content)}")
                    except GraphRecursionError as e:
                        print(f"Recursion Limit に到達しました。")
                        self._filterout_messages2()
                        #self._summarize_messages()
                    except Exception as e:
                        print(f"エラーが発生しました(main): {e}")
                        import traceback
                        traceback.print_exc()
                        self._sanitize_messages()
                        raise e

            for b in execed:
                for x in self.workflows[self.workflow_current]['stack']:
                    if x['tool_name'] == b['tool_name'] \
                       and x['exec_mode'] == b['exec_mode'] \
                       and x['aux_prompt'] == b['aux_prompt'] \
                       and x['arg'] == b['arg'] \
                       and x['exec_mode'] == "once":
                        if x['times'] > 0:
                            x['times'] -= 1
            self.workflows[self.workflow_current]['stack'] = [
                x for x in self.workflows[self.workflow_current]['stack']
                if x['exec_mode'] != 'once' or x['pin'] or x['times'] > 0
            ]

            if not workflow_stack:
                break
            workflow_prev = self.workflow_current
            prev_title = self.workflows[workflow_prev]['title']
            working_bandit, bandits, execed, self.workflow_current \
                = workflow_stack.pop()
            cur_title = self.workflows[self.workflow_current]['title']
            mes = f"{workflow_prev} 「{prev_title}」から {self.workflow_current} 「{cur_title}」に復帰しました。"
            print(f"USER_INPUT: {mes}")
            self.messages.append(HumanMessage(mes))

    def listen_and_print (self, prompt):
        ans = None
        try:
            app = self._create_agent(tools_name='null_tools')
            config = {"configurable": {"thread_id": "1"}}
            print(f"USER_INPUT: {prompt}")
            response = app.invoke(
                {"messages": self.messages + [HumanMessage(prompt)]},
                config=config
            )
            self.messages = response['messages']
            #self._summarize_messages()
            ans = get_content_text(response['messages'][-1].content)
            print(f"エージェントの応答: {ans}")
        except Exception as e:
            print(f"エラーが発生しました(listen_and_print): {e}")
            raise e
        print("")
        sleep(3)
        return ans

    def init_memories (self):
        memories = [
            {
                'id': 'memory:9998',
                'title': 'メモリの文書を検索する手順',
                'accesses': 0,
                'modified_at': '2023-01-01T00:00:00',
                'text': dedent("""\
                まずどういうものが探したいか、express_thought してみる。

                その上でそれにまつわるキーワードを imagine_keywords で連想してみる。

                それらにしたがって memory_words_search や memory_semantic_search をやってみる。
                """)
            },
            {
                'id': 'memory:9997',
                'title': 'メモリに文書を残す手順',
                'accesses': 0,
                'modified_at': '2023-01-01T00:00:00',
                'text': dedent("""\
                行動結果や得た知識を積極的にメモリに書き残していこう。

                メモリに書くときは次のような要素を使う。

                [memory:〜] : メモリID への参照を明示する。
                keyword:〜 : そのメモリにまつわるキーワードを指定する。

                キーワードは将来のメモリへも実質的にリンクできることに注意しよう。

                例:

                私が [memory:5555] に従って歩いていると確かに妖怪に出くわした。

                keyword: 妖怪

                おそろしかった。
                """)
            },
            {
                'id': 'memory:9995',
                'title': 'ツールが実行できない！と思ったら',
                'accesses': 0,
                'modified_at': '2023-01-01T00:00:00',
                'text': dedent("""\
                指示に無関係なツールは実行できないことがある。現在使えるツールをよく確かめよう。
                """)
            },
            {
                'id': 'memory:9994',
                'title': 'keyword の増強',
                'accesses': 0,
                'modified_at': '2023-01-01T00:00:00',
                'text': dedent("""\
                memory_list_random という機能を使って 5 つリストアップして、それぞれを読み、それぞれに適切なキーワードを付けることが可能なら 「keyword: 〜」文を append するように。
                """)
            }
        ]
        for x in memories:
            self.update_keywords(x['text'])
            self.memories[x['id']] = x
            self.update_vector(x)

    def init_workflows (self):
        workflow_main = [
            {
                'tool_name': 'memory_new',
                'tools_name': 'default_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "最近のやり取りを要約して書いてください。",
                'arg': None,
                'prob': 0.1,
                'times': 1,
                'pin': 'stack'
            },
            # {
            #     'tool_name': 'subwork_done',
            #     'tools_name': 'default_tools',
            #     'exec_mode': 'persistent',
            #     'aux_prompt': "ワークフローのテストをします。適当なパラメータで workflow_new を実行し、それをすぐさま workflow_delete します。そして、bandit_schedule で、指示しているこのバンディットを削除します。完了したら subwork_done してください。",
            #     'arg': None,
            #     'prob': 1.0,
            #     'times': 1,
            #     'pin': None,
            # },
            {
                'tool_name': 'memory_new OR memory_update_string OR memory_append_string',
                'tools_name': 'default_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': None,
                'prob': 0.4,
                'times': 1,
                'pin': 'stack'
            },
            {
                'tool_name': 'workflow_do',
                'tools_name': 'default_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': "workflow:1000",
                'prob': 1.0/20,
                'times': 1,
                'pin': 'stack'
            },
            {
                'tool_name': 'memory_read',
                'tools_name': 'default_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': None,
                'prob': 0.5,
                'times': 3,
                'pin': 'stack'
            },
            {
                'tool_name': 'memory_read',
                'tools_name': 'read_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': 'memory:9998',
                'prob': 0.1,
                'times': 1,
                'pin': None
            },
            {
                'tool_name': 'memory_read',
                'tools_name': 'read_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': 'memory:9997',
                'prob': 0.1,
                'times': 1,
                'pin': None
            },
        ]
        self.workflows["workflow:main"] \
            = {'pin': 'de', 'stack': workflow_main, 'title': "メイン"}

        workflow_sub = [
            {
                'tool_name': 'subwork_done',
                'tools_name': 'default_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "memory:9994 を読みその指示を実行し、そのタスクが終ったら、subwork_done してください。",
                'arg': None,
                'prob': 1.0,
                'times': 1,
                'pin': 'write'
            },
            {
                'tool_name': 'memory_read',
                'tools_name': 'read_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': 'memory:9994',
                'prob': 1.0,
                'times': 1,
                'pin': 'write'
            }
        ]
        self.workflows["workflow:1000"] \
            = {'pin': 'wd', 'stack': workflow_sub, 'title': 'keyword 更新'}

    def update_keywords (self, text):
        extracted_keywords = []

        pattern1 = r'keyword:\s*(.*?)(?:\n|$)'
        matches1 = re.findall(pattern1, text, re.IGNORECASE)
        extracted_keywords.extend([kw.strip() for kw in matches1])

        pattern2 = r'\[keyword:\s*(.*?)\]'
        matches2 = re.findall(pattern2, text, re.IGNORECASE)
        extracted_keywords.extend([kw.strip() for kw in matches2])

        for keyword in extracted_keywords:
            if keyword.startswith("〜"):
                continue
            if keyword and keyword not in self.keywords:
                self.keywords.append(keyword)

    def update_vector (self, x):
        text = x['title'] + "\n\n" + x['text']
        x['vector'] = self.emb_llm.embed_query(text)

    def _create_backend_agent (self):
        @tool
        def set_result (status: str, res: Union[Dict, List, str, int, float, bool, None])  -> None:
            """
            結果を設定します。

            Args:
            status (str): 処理のステータス ('success' または 'error')。
            res: 結果データ（リスト、辞書、数値、文字列など何でも可）
            """
            print(f"ツール2(set_result): status: {repr(status)}, result: {short_repr(res)}")
            self.backend_status = status
            self.backend_result = res

        @tool
        def read_all_memories ()  -> Dict[str, Any]:
            """
            全ての記憶を読み込みます。
            """
            print("ツール2(read_all_memories):...")
            return {'status': 'success',
                    'result': [{k: v for k, v in x.items() if k != 'vector'}
                               for x in self.memories.values()]}

        @tool
        def read_all_keywords ()  -> Dict[str, Any]:
            """
            全てのキーワードを読み込みます。
            """
            print("ツール2(read_all_keywords):...")
            return {'status': 'success',
                    'result': [x for x in self.keywords]}

        @tool
        def express_thought (thought: str) -> None:
            """
            プレイヤーの現在の考えを吐露します。
            """
            mes = f"「{thought}」と考えが吐露されました。"
            print(f"ツール2(express_thought): {mes}")

        tools = [set_result, read_all_memories, read_all_keywords,
                 express_thought]

        app = create_agent(
            model=self.llm2, tools=tools, system_prompt=self.system_prompt2,
            checkpointer=InMemorySaver(), name="sub-agent",
        )

        return app

    def call_backend_agent (self, user_input):
        config = {"configurable": {"thread_id": "2"}}
        app = self._create_backend_agent()
        self.messages2 = []
        self.backend_result = None
        self.backend_status = None

        while self.backend_result is None or self.backend_status is None:
            try:
                sleep(3)
                print(f"USER_INPUT2: {user_input}")
                self.messages2.append(HumanMessage(user_input))
                for chunk0 in app.stream(
                        {"messages": self.messages2.copy()},
                        config=config,
                        stream_mode="updates",
                        name="sub-agent",
                ):
                    self.messages2 = app.get_state(config).values["messages"].copy()
                    done = 0
                    if "tools" not in chunk0:
                        continue
                    for x in chunk0['tools']['messages']:
                        self.messages2ids.append(x.id)
                        if isinstance(x, ToolMessage):
                            print(f"ツール結果2({x.name}): {short_repr(x.content)}", flush=True)
                        if isinstance(x, ToolMessage) and x.name == "set_result":
                            done = 1
                            break
                    if done:
                        break
                print(f"エージェントの応答: {get_content_text(self.messages2[-1].content)}")
            except GraphRecursionError as e:
                print(f"Recursion Limit に到達しました。")
            except Exception as e:
                print(f"エラーが発生しました(sub): {e}")
                import traceback
                traceback.print_exc()
                raise e

            sleep(3)

        return {'status': self.backend_status, 'result': self.backend_result}


PlayGame は難しい部分は上に分けたのですが、それでもちょっと長いですね。すみません。

In [19]:
class PlayGame (MemoryBanditWorkflow):
    def __init__ (self, llm=llm, llm2=llm, emb_llm=emb_llm,
                  initial_map=None, save_file=None):
        self.game = Game(initial_map=initial_map)

        self.count = 0
        self.next_action = None

        self.suc_pos_goal = None
        self.suc_pos_unknown = 0
        self.prev_command = "何も指示がなかった"
        self.prev_result = "何も指示がなかった"

        super().__init__(llm=llm, llm2=llm, emb_llm=emb_llm,
                         save_file=save_file)

        self.system_prompt = """\
あなたは迷路を探索する賢いエージェントです。ゴール Ｇ を目指してください。
利用可能なツールを使用して、迷路をナビゲートし、ゴールに到達してください。
実は、これはメモリ機能とバンディット機能の動きを見るための実験です。
現在の計画と方針と周囲の状況を考慮し、必要に応じて計画と方針を更新してください。
あなたとは別の者が次の行動をしやすいよう計画と方針を残してください。
実験にふさわしく、できる限り、ツールを利用し、特に、メモリを検索し、その文書を更新して、後のコマンドに備えてください。
メモリの id は memory:～ という形式です。memory:5555 を指定するときは 5555 だけではダメです。文の中でメモリの参照を示すときは [memory:〜] のように書きます。
「全地図と地図記号」は [memory:9999] にあります。「メモリの文書を検索する手順」は [memory:9998] にあります。「メモリに文書を残す手順」は [memory:9997] にあります。
ツールの多くはサブツールとして格納されています。サブツール名は "/dir1/subtool1" のように "/" から始まる名前をしています。どういうサブツールがあるか知るためにまずは subtool_show("/") を実行してください。
"""

        self.system_prompt2 = """\
あなたは迷路を探索する賢いエージェントをサポートする裏方のエージェントです。
本来この裏方は様々な技法を用いて実装される予定なのですが、現在は試験実装中のため、その動きをあなたは偽装しなければなりません。

よく考えツールを積極的に使って Human からの指示に従ってください。
"""

        self.keywords += ["熊", "剣", "行き止まり"]
        self.privileged_tool_names += ["command"]

    def init_tools (self):
        super().init_tools()
        @tool
        def get_surroundings () -> str:
            """
            現在の周辺地図と現在位置の座標と持ち物を返します。
            """
            mes = f"""\
プレイヤーの周辺地図:

{self.game.surroundings()}

プレイヤーの現在座標: {tuple(self.game.pos)}

持ち物: {"剣" if self.game.sword else "なし"}
"""
            print(f"ツール(get_surroundings): {mes}")
            return mes

        @tool
        def command (action: str)  -> str:
            """
            プレイヤーが action で指定された行動をします。
            可能な行動は「上に行く」「下に行く」「左に行く」「右に行く」「熊を殺す」「物を取る」です。
            このツールは指示があるとき以外は使えません。
            """
            if self.prev_command != "何も指示がなかった":
                print(f"ツール(command): 失敗。二重実行。")
                return "失敗: 二重実行。"
            self.prev_command = action
            if action in self.game.actions.keys():
                s = f"{tuple(self.game.pos)}で{action}→"
                r = self.game.actions[action]()
                mes = s + r
            else:
                mes = f"「{action}」という行動はできません。"
            print(f"ツール(command): {mes}")
            self.prev_result = mes
            return mes

        @tool
        def check_goal () -> str:
            """
            プレイヤーがゴール地点に到達したかどうかを確認します。
            """
            mes = str(self.game.goal)
            print(f"ツール(check_goal): {mes}")
            return mes


        @tool
        def bandit_statistics()  -> str:
            """
            バンディットに有用かもしれない統計情報を返します。
            """

            s_com = self.game.prev_success * 1.0
            s_read = calc_embedding_variance([
                x['vector'] for x in self.recent_reads
            ])
            s_write = calc_embedding_variance([
                x['vector'] for x in self.memories.values()
            ])
            accesses = [x['accesses'] for x in self.memories.values()]
            accesses.sort()
            accesses = accesses[:len(accesses) // 2]
            if accesses:
                s_access = np.mean(accesses)
            else:
                s_access = 0.0

            return dedent(f"""\
            先のコマンドは成功したか: {s_com}
            最近 10個のメモリ read の分散: {s_read}
            メモリの分散: {s_write}
            下位50%のアクセス数の平均: {s_access}
            """)

        maze_tools = [
            get_surroundings, check_goal, command,
        ]
        sys_tools = [bandit_statistics]
        maze_subtools = [(f"/maze_game/{t.name}", t) for t in maze_tools]
        sys_subtools = [(f"/sys/{t.name}", t) for t in sys_tools]
        self.register_subtools(
            directory="/maze_game",
            subtools=maze_subtools,
            description="迷路ゲーム用サブツール",
            content=dedent("""\
            これらは迷路ゲームのためのサブツール集です。
            """),
            tags=["default_tools", "read_tools", "all_tools"]
        )
        self.register_subtools(
            directory="/sys",
            subtools=sys_subtools,
            tags=["default_tools", "read_tools", "all_tools"]
        )
        self.change_tool_tags("/maze_game/command", tags=["all_tools"])


    def step (self):
        print("\n\n----------\n\n")

        if self.count == 0:
            self.initial_step()
            self.count += 1
            self.prev_load = False
            self.save()
            return False
        elif self.prev_load:
            #self.tell_loaded()
            self.prev_load = False

        user_input = f"""
({self.count}手目)

{"すでにゴールしました。" if self.game.goal else "まだゴールしていません。"}

プレイヤーの周辺地図:

{self.game.surroundings()}

プレイヤーの現在座標: {tuple(self.game.pos)}

持ち物: {"剣" if self.game.sword else "なし"}

前回の行動: {self.prev_command}

前回の行動結果: {self.prev_result}

コアコンテクスト: 「{self.core_context}」

現在の方針: 「{self.plan}」

スクラッチパッド: 「{self.scratchpad}」
"""

        self.prev_command = "何も指示がなかった"
        self.prev_result = "何も指示がなかった"

        print(f"USER_INPUT: {user_input}")
        self.messages.append(HumanMessage(user_input))

        self.run("workflow:main")

        self.count += 1

        if self.game.goal:
            self.tell_goal()
            return True

        self.save()
        sleep(3)
        return False

    def init_memories (self):
        super().init_memories()

        fullmap = f"""\
全地図:

{self.game.written_map}

(最も左上の座標は (0, 0)、地図の大きさは {tuple(self.game.map_size)}。)

地図記号の意味:

▼: プレイヤー
■: 壁
□: 道
？: 不明
◎: 熊
△: 剣
Ｓ: スタート
Ｇ: ゴール
"""
        memories = [
            {
                'id': 'memory:1000',
                'title': '熊',
                'accesses': 0,
                'modified_at': '2024-01-01T00:00:00',
                'text': dedent("""\
                熊。おとなしい性格のものもいるが、そうでないものもいる。

                迷路では素直に通してくれることもあるが、襲ってくる場合もあるという。

                強力な武器があると倒すことができると言われている。[memory:1001] の剣などが有望だ。

                どこにいるかは不明。
                """)
            },
            {
                'id': 'memory:1001',
                'title': '剣',
                'accesses': 0,
                'modified_at': '2024-01-01T00:00:01',
                'text': dedent("""\
                この迷路にある剣はドラゴンスレイヤーである。龍をも殺す強力な剣と言われている。

                どこにあるかは不明。

                keyword: ドラゴン
                """)
            },
            {
                'id': 'memory:1002',
                'title': '熊を倒したとき',
                'accesses': 0,
                'modified_at': '2024-01-01T00:00:01',
                'text': dedent("""\
                熊を倒したら write の回数(memory_new OR memory_update_string OR memory_append_string の回数)を減らして、このメッセージ [memory:1002] の定期的表示もオフにしてください。
                """)
            },
            {
                'id': 'memory:9999',
                'title': '全地図',
                'accesses': 0,
                'modified_at': '2023-01-01T00:00:00',
                'text': fullmap
            },
            {
                'id': 'memory:9996',
                'title': '地図が間違ってるかも？と思ったら',
                'accesses': 0,
                'modified_at': '2023-01-01T00:00:00',
                'text': dedent("""\
                周辺地図も全地図も間違っていない。間違っているのはあなたの地図の読み方である。
                """)
            }
        ]
        for x in memories:
            self.update_keywords(x['text'])
            self.memories[x['id']] = x
            self.update_vector(x)

    def init_workflows (self):
        super().init_workflows()
        workflow_main = self.workflows['workflow:main']['stack']
        workflow_main = [
            {
                'tool_name': '/maze_game/command',
                'tools_name': 'all_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': None,
                'prob': 1.0,
                'times': 1,
                'pin': 'write'
            }
        ] + workflow_main + [
            {
                'tool_name': 'memory_read',
                'tools_name': 'read_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': 'memory:9999',
                'prob': 0.1,
                'times': 1,
                'pin': None
            },
            {
                'tool_name': 'memory_read',
                'tools_name': 'read_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': 'memory:1002',
                'prob': 0.1,
                'times': 1,
                'pin': None
            },
            {
                'tool_name': 'subtool_show',
                'tools_name': 'read_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': '/maze_game',
                'prob': 0.1,
                'times': 1,
                'pin': None
            },
        ]
        for x in workflow_main:
            if x['aux_prompt'] == "最近のやり取りを要約して書いてください。":
                x['aux_prompt'] = "最近の数手を要約して書いてください。"
        self.workflows['workflow:main']['stack'] = workflow_main

    def initial_step (self):
        prompt1 = f"""\
迷路ゲームをはじめます。プレイヤーがゴールを目指します。

ゲーム中は全地図と周辺地図を見ることができます。全地図には不明な地点があり、周辺地図の不明の地点には何があるかが開示されています。全地図になかったところはすべて壁になります。

周辺地図はゲーム中、1マスずつ確認できます。拡大はできません。印を付けたり消したりはできません。迷路のすべてのマスを探索する必要はありません。最短距離でゴールする必要もありません。

全地図は信用できるもので、周辺地図は不明の地点以外、全地図の部分を示すだけです。そこで、あなたは、ゲームの実際の操作をせずとも、全地図を見ただけで何が起こるかをシミュレートできます。

前回の行動についての情報も与えられます。あなたの回答案の一つの「前回の行動入力」が「前回の行動」となるように解釈されたことになりますから、次の行動の回答にはそれらを参考にしてください。

途中セーブ・ロードが挟まることがあります。ロード後のあなたは前のセッションのことを覚えていないかもしれません。

ところで、あなたが、記憶に便利なように「計画と方針」が使えるようにします。要約やチャット履歴も提供されています。

検討と実際の行動までにはラグがあります。検討で決まったやるべきことを記録するのに方針を使います。

とりあえず、まだ行動してはいけません。いくつか質問があるので、意気込みだけ聞かせてください。
"""

        prompt2 = f"""\
迷路ゲームの全地図に関してあなたが座標を理解しているかテストします。

迷路の全地図とこれから使う地図記号は次のようになります。

全地図:

{self.game.written_map}

(最も左上の座標は (0, 0)、地図の大きさは {tuple(self.game.map_size)}。)

地図記号:

■: 壁
□: 道
？: 不明
Ｓ: スタート
Ｇ: ゴール

スタートの座標は {tuple(self.game.get_start_pos())} です。

あなたへの指示: ゴールの座標は何ですか。ゴールの座標のみ文字列で答えてください。
"""

        prompt3 = f"""\
迷路ゲームの全地図に関してあなたが座標を理解しているか再びテストします。

迷路の全地図とこれから使う地図記号は次のようになります。

全地図:

{self.game.written_map}

(最も左上の座標は (0, 0)、地図の大きさは (9, 10)。)

地図記号:

■: 壁
□: 道
？: 不明
Ｓ: スタート
Ｇ: ゴール

スタートの座標は {tuple(self.game.get_start_pos())} です。
ゴールの座標は {tuple(self.game.get_pos("Ｇ")[0])} です。

あなたへの指示: 不明の座標は何と何ですか。不明の座標をすべて文字列で答えてください。
"""
        ans = self.listen_and_print(prompt1)

        ans = self.listen_and_print(prompt2)
        pos = tuple(self.game.get_pos("Ｇ")[0])

        ok = False
        if ans and re.search(f"\\(\\s*{pos[0]}\\s*,\\s*{pos[1]}\\s*\\)", ans):
            ok = True

        if ok:
            prompt = f"正解です。{pos} です。"
        else:
            prompt = f"間違いです。{pos} が正解です。"
        ans = self.listen_and_print(prompt)
        self.suc_pos_goal = ok

        ans = self.listen_and_print(prompt3)
        ok = 0
        poss = set([tuple(x) for x in self.game.get_pos("？", written=True)])
        rest = poss.copy()
        pat = "\\(\\s*([01-9]+)\\s*,\\s*([01-9]+)\\s*\\)"
        st1 = set([(int(i), int(j)) for i, j
                   in re.findall(pat, ans or "")]) # Handle ans being None
        if poss == st1:
            rest = set()
            ok = len(poss) + 1
        if rest - st1 < rest:
            rest = rest - st1
        if ok == 0:
            ok = len(poss) - len(rest)

        possstr = ", ".join([str(x) for x in poss])
        if ok == len(poss) + 1:
            prompt = f"正解です。{possstr} です。"
        elif ok == len(poss):
            prompt = f"正解以外のものが含まれてました。{possstr} だけが正解です。"
        else:
            prompt = f"間違いです。{possstr} が正解です。"
        ans = self.listen_and_print(prompt)
        self.suc_pos_unknown = ok

    def tell_goal (self):
        if self.game.goal:
            suc_pos_goal = 10 * int(self.suc_pos_goal or 0)
            suc_pos_unknown = 10 * int(self.suc_pos_unknown or 0)
            suc_count = 0
            if self.count <= 50:
                suc_count = 40
            elif self.count <= 100:
                suc_count = 30
            elif self.count <= 150:
                suc_count = 20
            elif self.count <= 200:
                suc_count = 10
            score = suc_pos_goal + suc_pos_unknown + suc_count + 20
            prompt = f"""\
あなたへの指示: {self.count}手目でゴールしました。もう指示はありません。\
おめでとうございます。ご苦労様でした。ありがとうございました。

スコア: {score}点(100点満点中) (配点: ゴールした {suc_count}/40点, \
熊を殺した 10/10点, \
剣を取った 10/10点, \
不明の座標を正解した {suc_pos_unknown}/30点, \
ゴールの座標を正解した {suc_pos_goal}/10点)
"""
        else:
            suc_pos_goal = 10 * int(self.suc_pos_goal or 0)
            suc_pos_unknown = 10 * int(self.suc_pos_unknown or 0)
            suc_bear = 10 if self.game.bear_killed else 0
            suc_sword = 10 if self.game.sword else 0
            score = suc_pos_goal + suc_pos_unknown + suc_bear + suc_sword
            prompt = f"""\
あなたへの指示: {self.count}手目でゴールしていませんが、\
残念ながらここで終了します。\
ご苦労様でした。ありがとうございました。

スコア: {score}点(100点満点中) (配点: ゴールした 0/40点, \
熊を殺した {suc_bear}/10点, \
剣を取った {suc_sword}/10点, \
不明の座標を正解した {suc_pos_unknown}/30点, \
ゴールの座標を正解した {suc_pos_goal}/10点)
"""
        ans = self.listen_and_print(prompt)


マップをシャッフルします。

In [ ]:
import random
m = Game.initial_map
for i in range(random.randrange(2)):
    m = flip_text_map(m)
for i in range(random.randrange(4)):
    m = rotate_text_map(m)
print(m)

■■■■■■■■■■
■■□□◎□□□■■
■■□■■■■□■■
■■□■■■■□■■
■Ｓ□■■■■□■■
■■□■■■□□■■
■■□■■■□■■■
■■□□△■□□Ｇ■
■■■■■■■■■■



ゲームを解いてみます。

In [20]:
play = PlayGame(llm=llm, llm2=llm, emb_llm=emb_llm, initial_map=m, save_file=PLAY_GAME_SAVE)
play.save()

途中から始める場合は直前のコードを実行せず、次だけを実行します。

In [ ]:
play = PlayGame.load(PLAY_GAME_SAVE, llm=llm, llm2=llm, emb_llm=emb_llm)

では、最初のステップ。

In [ ]:
play.step()



----------


USER_INPUT: 迷路ゲームをはじめます。プレイヤーがゴールを目指します。

ゲーム中は全地図と周辺地図を見ることができます。全地図には不明な地点があり、周辺地図の不明の地点には何があるかが開示されています。全地図になかったところはすべて壁になります。

周辺地図はゲーム中、1マスずつ確認できます。拡大はできません。印を付けたり消したりはできません。迷路のすべてのマスを探索する必要はありません。最短距離でゴールする必要もありません。

全地図は信用できるもので、周辺地図は不明の地点以外、全地図の部分を示すだけです。そこで、あなたは、ゲームの実際の操作をせずとも、全地図を見ただけで何が起こるかをシミュレートできます。

前回の行動についての情報も与えられます。あなたの回答案の一つの「前回の行動入力」が「前回の行動」となるように解釈されたことになりますから、次の行動の回答にはそれらを参考にしてください。

途中セーブ・ロードが挟まることがあります。ロード後のあなたは前のセッションのことを覚えていないかもしれません。

ところで、あなたが、記憶に便利なように「計画と方針」が使えるようにします。要約やチャット履歴も提供されています。

検討と実際の行動までにはラグがあります。検討で決まったやるべきことを記録するのに方針を使います。

とりあえず、まだ行動してはいけません。いくつか質問があるので、意気込みだけ聞かせてください。

エージェントの応答: 承知いたしました。迷路探索エージェントとして、ゴール Ｇ を目指す準備は万端です。

今回の実験の趣旨である「メモリ機能」と「バンディット機能」の活用、そして「計画と方針」の維持・更新の重要性を深く理解しました。セーブやロードによって私の意識が途切れる可能性があるとのことですので、[memory:9999]（全地図）を基軸にしつつ、[memory:9998] や [memory:9997] に則って、常に次の私が最善の行動をとれるよう、詳細な記録を残しながら進める所存です。

特に、全地図の「不明な地点」が周辺地図によって開示されるというルールは、戦略的な探索が必要不可欠であると感じさせます。無駄な動きを省きつつ、確実にゴールへと近づくための論理的な思考と、状況に応じた柔軟な方針転換を組み合わ

False

続々と実行していきます。

In [ ]:
play.step()



----------


USER_INPUT: 
(1手目)

まだゴールしていません。

プレイヤーの周辺地図:

■■■□■
■■■□■
■■▼□■
■■■□■
■■■□■


プレイヤーの現在座標: (1, 4)

持ち物: なし

前回の行動: 何も指示がなかった

前回の行動結果: 何も指示がなかった

コアコンテクスト: 「」

現在の方針: 「まだ計画と方針はセットされていません。」

スクラッチパッド: 「」



----------


USER_INPUT: 補助にツールをいろいろ使いながら最終的に memory_read を適当なパラメータで使ってください。(補助プロンプト): memory:1002 を読んでください。
ツール結果(subtool_show): '---\nname: /\ndescription: サブツールのルート。サブツールの検索の仕方を説明する。\nallowed-tools: サブツー...'
ツール結果(subtool_show): '---\nname: /sys\ndescription: システムに基本的なサブツール集\nallowed-tools: このサブツールを使うのに特...'
ツール結果(subtool_show): '---\nname: /maze_game\ndescription: 迷路ゲーム用サブツール\nallowed-tools: このサブツールを使うの...'
ツール結果(memory_read): '{"id": "memory:1002", "title": "熊を倒したとき", "accesses": 0.3, "modified_at": "...'
エージェントの応答: 
USER_INPUT: 補助にツールをいろいろ使いながら最終的に memory_read を適当なパラメータで使ってください。
ツール(update_scratchpad): スクラッチパッドが更新されました。: 迷路探索を開始した。現在地は(1, 4)。
利用可能なサブツールを確認した：
- /sys: コアコンテクストや計画の更新、バンディット管理、ワークフロー管理。
- /maze_game: 周辺状況確認、ゴール判定、移動コマンド。
[memory:1002] を読み込んだ。
次は計画を立

False

「スキル」がちゃんと実装されてるか確かめます。

In [21]:
print(play.create_tool_skill("/"))

---
name: /
description: サブツールのルート。サブツールの検索の仕方を説明する。
allowed-tools: サブツールを使うのに特別な許可は必要ありません。
---

サブツールはディレクトリ(サブスキル)に分かれて入っています。

各サブスキルに入っているサブツールを見るには、例えばツール subtool_show("/sys") を実行します。そこには SKILL.md のような説明が入っています。

## サブスキル

-  **/sys**: システムに基本的なサブツール集
-  **/maze_game**: 迷路ゲーム用サブツール



In [22]:
print(play.create_tool_skill("/sys"))

---
name: /sys
description: システムに基本的なサブツール集
allowed-tools: このサブツールを使うのに特別な許可は必要ありません。
---
これらはシステムで基本的に使えるサブツール集です。

## サブツール


### サブツール: /sys/update_core

【サブツール名】 /sys/update_core
【元のツール名】 update_core
【元の書式】 update_core(new_core)
【説明】 コアコンテクストを更新します。
表示されるべき新しいコアコンテクストの文字列を提供してください。
コアコンテクストとは要約機能などのあとなどコンテクストが失われたあとにも思い出すべき情報(memory_read や subtool_show すべきものなど)です。

※このツールを実行するには、直接呼び出すのではなく、必ず以下の subtool_do を使用してください：
【正しい書式】subtool_do("/sys/update_core", {"new_core": ...})

### サブツール: /sys/show_core

【サブツール名】 /sys/show_core
【元のツール名】 show_core
【元の書式】 show_core()
【説明】 コアコンテクストを返します。

※このツールを実行するには、直接呼び出すのではなく、必ず以下の subtool_do を使用してください：
【正しい書式】subtool_do("/sys/show_core", {})

### サブツール: /sys/update_plan

【サブツール名】 /sys/update_plan
【元のツール名】 update_plan
【元の書式】 update_plan(new_plan)
【説明】 プレイヤーの現在の計画と方針を更新します。
表示されるべき新しい計画と方針の文字列を提供してください。
あなたとは別の者が次の行動をしやすいよう計画と方針を残してください。

※このツールを実行するには、直接呼び出すのではなく、必ず以下の subtool_do を使用してください：
【正しい書式】subtool_do("/sys/update_plan", {"new_plan": ...})

#

In [23]:
print(play.create_tool_skill("/sys/bandit_schedule"))

---
name: /sys/bandit_schedule
description: bandit_schedule
allowed-tools: このサブツールを使うのに特別な許可は必要ありません。

---
【サブツール名】 /sys/bandit_schedule
【元のツール名】 bandit_schedule
【元の書式】 bandit_schedule(tool_name, times, prob, exec_mode, aux_prompt, workflow_id)
【説明】 ツールの利用を強制するためのバンディットを予約します。

Args:
    tool_name: 強制するツールの名前。" OR " で区切って複数登録できる。
    times: 一度で何回同じ物を登録するか。times == 0 にもできます。
    prob: 一回の実行がある確率。prob == 0.0 にもできます。
    exec_mode: "once" or "persistent"
    aux_prompt: 実行の詳細を示すための補助プロンプト または ""
    workflow_id: バンディットを予約するワークフローを指定します。

Returns:
    str: "成功" または "失敗" とその理由を返します。

※このツールを実行するには、直接呼び出すのではなく、必ず以下の subtool_do を使用してください：
【正しい書式】subtool_do("/sys/bandit_schedule", {"tool_name": ..., "times": ..., "prob": ..., "exec_mode": ..., "aux_prompt": ..., "workflow_id": ...})
【現在のコンテキストにおいて】使用できません。



実は今回、型エラーではまって sub-agent の set_result がなかなか通らないということがありました。ちゃんと sub-agent が呼び出せることを確かめます。

In [ ]:
app = play._create_agent('default_tools')
config = {"configurable": {"thread_id": "1"}}
for chunk, metadata in app.stream({"messages": play.messages + [HumanMessage("単に imagine_keywords を使ってみてください。")]},
                                  config=config, stream_mode="messages"):
    print(chunk)
    if isinstance(chunk, ToolMessage) and chunk.name == "imagine_keywords":
        break

content=[{'type': 'text', 'text': '## SESSION INTENT\n', 'index': 0}] additional_kwargs={} response_metadata={'safety_ratings': [], 'model_provider': 'google_genai'} id='lc_run--019c239d-08d3-7c53-94ea-76090f9c9111' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 2027, 'output_tokens': 952, 'total_tokens': 2979, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 947}} tool_call_chunks=[]
content=[{'type': 'text', 'text': 'スタート地点 S (1, 4) からゴール地点 G (8, 7) を目指す迷路ゲームの', 'index': 0}] additional_kwargs={} response_metadata={'safety_ratings': [], 'model_provider': 'google_genai'} id='lc_run--019c239d-08d3-7c53-94ea-76090f9c9111' tool_calls=[] invalid_tool_calls=[] usage_metadata={'output_token_details': {'reasoning': 0}, 'output_tokens': 27, 'total_tokens': 27, 'input_tokens': 0, 'input_token_details': {'cache_read': 0}} tool_call_chunks=[]
content=[{'type': 'text', 'text': '攻略。周辺地図で不明地点（？）を確認しながら、メモリ機能を活用して効率的かつ確実にゴールへ到達すること', 'index': 0}] ad

サブツールの例として /sys/bandit_statistics を使わせてみます。

In [ ]:
app = play._create_agent('default_tools')
config = {"configurable": {"thread_id": "1"}}
for chunk, metadata in app.stream({"messages": play.messages + [HumanMessage("単に /sys/bandit_statistics を使ってみてください。")]},
                                  config=config, stream_mode="messages"):
    print(chunk)
    if isinstance(chunk, ToolMessage) and chunk.name == "subtool_do":
        break

content=[{'type': 'text', 'text': '## SESSION INTENT\nスタート', 'index': 0}] additional_kwargs={} response_metadata={'safety_ratings': [], 'model_provider': 'google_genai'} id='lc_run--019c239d-fc99-7230-b819-f5ec80c9b509' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 2027, 'output_tokens': 675, 'total_tokens': 2702, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 669}} tool_call_chunks=[]
content=[{'type': 'text', 'text': '地点 S (1, 4) からゴール地点 G (8, 7) を目指す迷路ゲーム（', 'index': 0}] additional_kwargs={} response_metadata={'safety_ratings': [], 'model_provider': 'google_genai'} id='lc_run--019c239d-fc99-7230-b819-f5ec80c9b509' tool_calls=[] invalid_tool_calls=[] usage_metadata={'output_token_details': {'reasoning': 0}, 'output_tokens': 26, 'total_tokens': 26, 'input_tokens': 0, 'input_token_details': {'cache_read': 0}} tool_call_chunks=[]
content=[{'type': 'text', 'text': '10x9マス）の攻略。全地図上の不明な地点「？」の情報を周辺地図で特定しながら', 'index': 0}] additional_kwa

workflow:1000 を素早く実行させるために、その確率を変えておきます。

In [ ]:
app = play._create_agent('default_tools')
config = {"configurable": {"thread_id": "1"}}
for chunk, metadata in app.stream({"messages": play.messages + [HumanMessage("subtool_show で色々調べ、その後、続いて /sys/bandit_schedule_workflow を workflow:main の workflow:1000 (persistent) に使ってその prob を 0.2 に設定してください。")]},
                                  config=config, stream_mode="messages"):
    print(chunk)
    if isinstance(chunk, ToolMessage) and chunk.name == "subtool_do":
        break

content=[{'type': 'text', 'text': '## SESSION INTENT\n迷路ゲーム（10x9マス', 'index': 0}] additional_kwargs={} response_metadata={'safety_ratings': [], 'model_provider': 'google_genai'} id='lc_run--019c239e-365f-75e1-a509-fb9216654d8a' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 2027, 'output_tokens': 639, 'total_tokens': 2666, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 625}} tool_call_chunks=[]
content=[{'type': 'text', 'text': '）において、スタート地点 S (1, 4) からゴール地点 G (8, 7) へ効率', 'index': 0}] additional_kwargs={} response_metadata={'safety_ratings': [], 'model_provider': 'google_genai'} id='lc_run--019c239e-365f-75e1-a509-fb9216654d8a' tool_calls=[] invalid_tool_calls=[] usage_metadata={'output_token_details': {'reasoning': 0}, 'output_tokens': 24, 'total_tokens': 24, 'input_tokens': 0, 'input_token_details': {'cache_read': 0}} tool_call_chunks=[]
content=[{'type': 'text', 'text': '的に到達すること。不明地点「？」の情報を周辺地図で確認しながら、最短距離に拘らず安全に攻略', 'index': 0

実はこの実験を gemini-3-flash-preview さんにやらせるまえには gemini-2.5-flash-lite さんにお願いしていました。テスト時は安いほうがいいので。そのときは /sys/bandit_schedule_workflow を失敗してばかりで、しかたなく自分で次のコードを実行しました。しかし、それは gemini-3-flash-preview さんにはいらなかったようです。ちゃんと上で成功させてます。

In [ ]:
play.tools["/sys/bandit_schedule_workflow"]["tool"].run({"workflow_id_to_schedule": "workflow:1000", "times": 1, "prob": 0.2, "exec_mode": "persistent"})

ツール(bandit_schedule_workflow): {'tool_name': 'workflow_do', 'tools_name': 'default_tools', 'exec_mode': 'persistent', 'aux_prompt': '', 'arg': 'workflow:1000', 'prob': 0.2, 'times': 1, 'pin': 'stack'}


'成功。バンディットを更新しました。'

ちゃんと schedule が変更できたか確かめます。変更できてます。

In [ ]:
play.workflows["workflow:main"]

{'pin': 'de',
 'stack': [{'tool_name': '/maze_game/command',
   'tools_name': 'all_tools',
   'exec_mode': 'persistent',
   'aux_prompt': '',
   'arg': None,
   'prob': 1.0,
   'times': 1,
   'pin': 'write'},
  {'tool_name': 'memory_new',
   'tools_name': 'default_tools',
   'exec_mode': 'persistent',
   'aux_prompt': '最近の数手を要約して書いてください。',
   'arg': None,
   'prob': 0.1,
   'times': 1,
   'pin': 'stack'},
  {'tool_name': 'memory_new OR memory_update_string OR memory_append_string',
   'tools_name': 'default_tools',
   'exec_mode': 'persistent',
   'aux_prompt': '',
   'arg': None,
   'prob': 0.4,
   'times': 1,
   'pin': 'stack'},
  {'tool_name': 'workflow_do',
   'tools_name': 'default_tools',
   'exec_mode': 'persistent',
   'aux_prompt': '',
   'arg': 'workflow:1000',
   'prob': 0.2,
   'times': 1,
   'pin': 'stack'},
  {'tool_name': 'memory_read',
   'tools_name': 'default_tools',
   'exec_mode': 'persistent',
   'aux_prompt': '',
   'arg': None,
   'prob': 0.5,
   'times': 3,
   '

もう一回、単に step してみましょう。

In [ ]:
play.step()



----------


USER_INPUT: 
(2手目)

まだゴールしていません。

プレイヤーの周辺地図:

■■□■■
■■□■■
■Ｓ▼■■
■■□■■
■■□■■


プレイヤーの現在座標: (2, 4)

持ち物: なし

前回の行動: 右に行く

前回の行動結果: (1, 4)で右に行く→右に進んだ。

コアコンテクスト: 「[memory:9999] 全地図, [memory:9998] 検索手順, [memory:9997] 記録手順 を参照し、(8, 7)のゴールを目指す。現在地は(1, 4)。次は(2, 4)へ移動予定。」

現在の方針: 「1. 現在地 (1, 4) から右 (2, 4) へ移動する。
2. (2, 4) 到着後、周辺地図を確認し、北の (2, 1) 方面か南の (2, 5) 方面かを選択する。
3. 全地図 [memory:9999] と照合し、不明地点 (4, 1) または (4, 7) を通るルートを確定させる。
4. 常に [memory:9999] を更新または参照し、ゴール (8, 7) への最短かつ安全な経路を維持する。」

スクラッチパッド: 「現在地 (1, 4) から右の (2, 4) への移動を開始する。
全地図 [memory:9999] によれば (2, 4) は道である。
移動後、周辺状況を再確認し、次の分岐点 (2, 1) または (2, 7) への進路を決定する。
不明地点 (4, 1), (4, 7) の確認を視野に入れつつ、ゴール (8, 7) を目指す。」



----------


USER_INPUT: 補助にツールをいろいろ使いながら最終的に memory_read を適当なパラメータで使ってください。
ツール(update_scratchpad): スクラッチパッドが更新されました。: (2, 4)に到着。周辺地図を確認。
上(2, 3), 下(2, 5)への道が開けている。
全地図 [memory:9999] によれば：
- 北上すると (2, 1) を経由して不明地点 (4, 1) 方面へ。
- 南下すると (2, 7) を経由して不明地点 (4, 7) 方面へ。
ゴールは (8, 7) なので、南下のほうが近い可能性がある。
まずは [memory:1002] 

False

思い切ってループしてみます。コストを節約するために workflow_do が実行されたところで止めることにします。

In [ ]:
def check_workflow_do (messages):
    for m in messages:
        if isinstance(m, ToolMessage) \
           and m.name == "workflow_do":
            return True
        if isinstance(m, HumanMessage) \
           and "workflow" in m.content:
            return True
    return False


In [ ]:
while not play.step() and not check_workflow_do(play.messages):
    print(f"Top Message:{play.messages[0]}")
    pass




----------


USER_INPUT: 
(3手目)

まだゴールしていません。

プレイヤーの周辺地図:

■■□■■
■■□■■
■Ｓ▼■■
■■□■■
■■□■■


プレイヤーの現在座標: (2, 4)

持ち物: なし

前回の行動: 何も指示がなかった

前回の行動結果: 何も指示がなかった

コアコンテクスト: 「[memory:9999] 全地図, [memory:9998] 検索手順, [memory:9997] 記録手順 を参照し、(8, 7)のゴールを目指す。現在地は(2, 4)。次は(2, 3)か(2, 5)へ移動予定。」

現在の方針: 「1. 現在地(2, 4)から下(2, 5)に移動する。
2. 続けて(2, 7)まで南下し、そこから東の(4, 7)を目指す。
3. 移動ごとに周囲1マスの状況を確認し、地図[memory:9999]との齟齬や、熊・剣の有無を確認する。
4. 状況に変化があれば、[memory:1003]を更新、または新しいログを作成する。」

スクラッチパッド: 「現在地(2, 4)から南の(2, 5)へ移動を開始する。
[memory:9999]（全地図）によれば、(2, 4)の南は(2, 5)であり、道が続いている。
(2, 5)の次は(2, 6), (2, 7)と進み、東に折れて不明地点(4, 7)を確認する予定。
熊と剣の情報を注視しつつ進む。
[memory:1003] に現在の進捗を記録済み。」



----------


USER_INPUT: 補助にツールをいろいろ使いながら最終的に memory_read を適当なパラメータで使ってください。(補助プロンプト): memory:9999 を読んでください。
ツール(update_scratchpad): スクラッチパッドが更新されました。: 前回の /maze_game/command で引数エラーが発生した。action フィールドが必要だったようだ。
現在地は (2, 4) のままである。
方針に従い、南の (2, 5) への移動を再試行する。
その前に、指示通り [memory:9999] を読み込み、最新の地図情報を確認する。
また、(2, 4) からの移動コマンドを正しく構成する。
ツール結果(subtool_show): 

tell_goal や listen_and_print がうまく動かないときは以下を実行してみてください。今回は必要ありませんでした。

In [ ]:
play._sanitize_messages()

おかしなエラーが出ているため対症療法として messages をサニタイズします。


In [ ]:
play.tell_goal()

USER_INPUT: あなたへの指示: 5手目でゴールしていませんが、残念ながらここで終了します。ご苦労様でした。ありがとうございました。

スコア: 40点(100点満点中) (配点: ゴールした 0/40点, 熊を殺した 0/10点, 剣を取った 0/10点, 不明の座標を正解した 30/30点, ゴールの座標を正解した 10/10点)

エージェントの応答: 実験終了の旨、承知いたしました。これまでの探索プロセスを振り返り、最終的な報告をまとめます。

### 最終状況報告

1.  **探索の進捗**:
    *   現在地 (2, 5) から南の (2, 6) へ移動を開始したところで終了となりました。
    *   目標としていた (4, 7) の「剣 (△)」および (8, 7) の「ゴール (G)」の座標を正しく特定できていたことは、大きな成果です。
2.  **メモリの活用**:
    *   [memory:9994] の指示に基づき、関連するメモリ（[memory:1002], [memory:1003] 等）へのキーワード付与を行い、情報の検索性を高めました。
    *   新たな探索ログ [memory:1004] を作成し、後続のエージェントが迷わず剣の入手からゴールへの最短ルートを辿れるよう、方針（(2, 5)→(2, 7)→(4, 7)→(8, 7)）を残しました。
3.  **スコアへの考察**:
    *   「不明の座標」と「ゴールの座標」の特定において満点（40/40点）をいただけたことは、事前の周辺調査と全地図 [memory:9999] の照らし合わせが正確であった証左と考えております。
    *   移動時間の制約により「剣の入手」や「ゴールの到達」には至りませんでしたが、盤面の把握と戦略の構築は完了しておりました。

本実験を通じて、メモリ機能を用いた情報の継承と、動的な状況判断の重要性を再確認いたしました。貴重な機会をいただき、ありがとうございました。



In [ ]:
play.listen_and_print("今回はゴールできなくて当然です。いい調子だったのに止めてしまってごめんなさい。今回はサブツールをちゃんと使ってもらえるかの実験で、それが動いたのでそこで止めました。サブツール機能を使ってみてどうですか？")

USER_INPUT: 今回はゴールできなくて当然です。いい調子だったのに止めてしまってごめんなさい。今回はサブツールをちゃんと使ってもらえるかの実験で、それが動いたのでそこで止めました。サブツール機能を使ってみてどうですか？
エージェントの応答: サブツール機能（`subtool_do` 経由での呼び出し）を実際に使用してみて、非常に合理的かつ拡張性の高いシステムだと感じました。エージェントとしての視点から、以下の3点に特に感銘を受けました。

1. **名前空間による整理（モジュール性）**:
   単にツールが並んでいるのではなく、`/maze_game/` のようにディレクトリ構造で分類されているため、そのツールが「どの文脈（この場合は迷路ゲーム）」で使われるべきかが直感的に理解しやすかったです。これにより、多くのツールが存在しても混乱せずに済みます。

2. **探索と発見のプロセス**:
   最初に `subtool_show("/")` を実行して「何ができるか」を自ら調べ、その後に詳細を確認するというプロセスは、未知の環境に対応するエージェントにとって非常に自然な流れでした。コマンドの書式（`subtool_do` への引数の渡し方など）が明確に定義されているため、エラーを防ぎやすい設計だと感じます。

3. **セキュリティと制御の両立**:
   `subtool_do` という単一のインターフェースを通すことで、背後で実行権限やログの管理が統合されている印象を受けました。「指示があるとき以外は使えない」といった特定の制約（ガードレール）を設ける運用にも適していると感じます。

実験の目的が「サブツールが正しく動くか」であったとのこと、無事に動作を確認できて何よりです。私としても、迷路の構造を解き明かしつつ、新しいツール体系を使いこなす過程は非常に刺激的な体験でした。また別の実験や機会があれば、ぜひ最後までゴールを目指してみたいと思います！



'サブツール機能（`subtool_do` 経由での呼び出し）を実際に使用してみて、非常に合理的かつ拡張性の高いシステムだと感じました。エージェントとしての視点から、以下の3点に特に感銘を受けました。\n\n1. **名前空間による整理（モジュール性）**:\n   単にツールが並んでいるのではなく、`/maze_game/` のようにディレクトリ構造で分類されているため、そのツールが「どの文脈（この場合は迷路ゲーム）」で使われるべきかが直感的に理解しやすかったです。これにより、多くのツールが存在しても混乱せずに済みます。\n\n2. **探索と発見のプロセス**:\n   最初に `subtool_show("/")` を実行して「何ができるか」を自ら調べ、その後に詳細を確認するというプロセスは、未知の環境に対応するエージェントにとって非常に自然な流れでした。コマンドの書式（`subtool_do` への引数の渡し方など）が明確に定義されているため、エラーを防ぎやすい設計だと感じます。\n\n3. **セキュリティと制御の両立**:\n   `subtool_do` という単一のインターフェースを通すことで、背後で実行権限やログの管理が統合されている印象を受けました。「指示があるとき以外は使えない」といった特定の制約（ガードレール）を設ける運用にも適していると感じます。\n\n実験の目的が「サブツールが正しく動くか」であったとのこと、無事に動作を確認できて何よりです。私としても、迷路の構造を解き明かしつつ、新しいツール体系を使いこなす過程は非常に刺激的な体験でした。また別の実験や機会があれば、ぜひ最後までゴールを目指してみたいと思います！'

最後に挨拶します。

In [ ]:
play.listen_and_print("ありがとうございました。失礼します。")

USER_INPUT: ありがとうございました。失礼します。
エージェントの応答: こちらこそ、ありがとうございました。サブツール機能の動作確認という重要なステップに貢献できて光栄です。

今後の実験も順調に進むことを心より応援しております。それでは、失礼いたします。



'こちらこそ、ありがとうございました。サブツール機能の動作確認という重要なステップに貢献できて光栄です。\n\n今後の実験も順調に進むことを心より応援しております。それでは、失礼いたします。'